In [ ]:
# Note that need to use pythonnet 2.3.*

In [ ]:
import sys,random,os,gc
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import pandas as pd
import arcpy
import ee
from scipy import stats
#import rpy2

#Set arcpy to overwrite output
arcpy.env.overwriteOutput = True

In [ ]:
from scipy import stats
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
import clr, System
sys.path.append(r".\code_libs\arcmet_code_libs_10.7.1")
clr.AddReference("System.Collections")
clr.AddReference("ArcMET_Base") #Import ArcMET_Base.dll (Movement Ecology Tools for ArcGIS - www.movementecology.net)
from ArcMET_Base import *

In [ ]:
# #Import AnimalTracking2_Esri_DB.dll
# clr.AddReference("AnimalTracking2_Esri_DB")
# from AnimalTracking2_Esri_DB import *

In [ ]:
#Import ArcGIS_EE_Utilities
from ArcGIS_EE_Utilities import *

### Global Variables

In [ ]:
rootDrive='C:\\Users\\jake_\\dropbox'
rootFolder = rootDrive + '\\Research_PanAfEl\\'
MovDataFolder = rootFolder + 'Movement Data'
RangeFolder =  rootFolder + 'Analyses\\Ranges'
etdFolder =  RangeFolder + '\\ETD'
mcpFolder =  RangeFolder + '\\MCP'
kdeFolder =  RangeFolder + '\\KDE'
tempmetricsFolder = rootFolder + 'Analyses\\TemporalMetrics'
trajPathMetricsFolder = rootFolder + 'Analyses\\TrajPathMetrics'
SpatialDataFolder = rootFolder + 'Spatial Data'
figuresFolder = rootFolder + 'Figures'
tablesFolder = rootFolder + 'Tables'

etd16DayAnalysisTable = tablesFolder + '\\ETD_16Day_Analysis_Table.csv'
etdAnnualAnalysisTable = tablesFolder + '\\ETD_Annual_Analysis_Table.csv'
etd16DayAnalysisTableFilt = tablesFolder + '\\ETD_16Day_Analysis_Table_Filtered.csv'
etdAnnualAnalysisTableFilt = tablesFolder + '\\ETD_Annual_Analysis_Table_Filtered.csv'
AnnualAnalysisTable = tablesFolder + '\\Annual_Analysis_Table.csv'

#Define a file with subject & dates that signal compromised data not to be included in analysis
ignoreDataFile = rootFolder + 'Analyses\\ignoreData.csv'

#AnimalTracking DB
animalTrackingDB_Path= MovDataFolder + "\\AnimalTracking.gdb"
tmTable = animalTrackingDB_Path + "\\TRACKINGMASTER"

#GCS Spatial Reference
wgs84GCS ="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]]" + \
",PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"

In [ ]:
modisStartDate = System.DateTime(2000,3,5,0,0,0) #This is the earliest valid MODIS imagery date
startDate = System.DateTime(1998,2,6,8,0,0) #The first datapoint in the database
endDate = System.DateTime(2014,1,1,0,0,0) #Run the analysis up to the end of 2013

#Define the desired range percentiles
percentiles = System.Collections.Generic.List[System.Double]()
percentiles.Add(System.Double(0.5))
percentiles.Add(System.Double(0.90))
percentiles.Add(System.Double(0.95))
percentiles.Add(System.Double(1.0))

In [ ]:
#Define the output database properties
MovGDB_Path = MovDataFolder + "\\MovData.gdb"
MovGDB_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
MovGDB_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
MovGDB_PropVals.Add(System.Tuple[str,str]("DATABASE", MovGDB_Path))
MovGDB_Props = GeodatabaseHelper.CreateConnection(MovGDB_PropVals)

In [ ]:
#Define the output databases/props for the 16-Day & Annual ETD Ranges
etdRanges16Day_Path = etdFolder + "\\ETD16DayRanges.gdb"
etdRanges16Day_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
etdRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
etdRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASE", etdRanges16Day_Path))
etdRanges16Day_Props = GeodatabaseHelper.CreateConnection(etdRanges16Day_PropVals)

etdRangesAnnual_Path = etdFolder + "\\ETDAnnualRanges.gdb"
etdRangesAnnual_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
etdRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
etdRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASE", etdRangesAnnual_Path))
etdRangesAnnual_Props = GeodatabaseHelper.CreateConnection(etdRangesAnnual_PropVals)

In [ ]:
#Define the output databases/props for the 16-Day & Annual KDE Ranges
kdeRanges16Day_Path = kdeFolder + "\\KDE16DayRanges.gdb"
kdeRanges16Day_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
kdeRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
kdeRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASE", kdeRanges16Day_Path))
kdeRanges16Day_Props = GeodatabaseHelper.CreateConnection(kdeRanges16Day_PropVals)

kdeRangesAnnual_Path = kdeFolder + "\\KDEAnnualRanges.gdb"
kdeRangesAnnual_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
kdeRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
kdeRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASE", kdeRangesAnnual_Path))
kdeRangesAnnual_Props = GeodatabaseHelper.CreateConnection(kdeRangesAnnual_PropVals)

In [ ]:
#Define the output databases/props for the 16-Day & Annual MCP Ranges
mcpRanges16Day_Path = mcpFolder + "\\MCP16DayRanges.gdb"
mcpRanges16Day_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
mcpRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
mcpRanges16Day_PropVals.Add(System.Tuple[str,str]("DATABASE", mcpRanges16Day_Path))
mcpRanges16Day_Props = GeodatabaseHelper.CreateConnection(mcpRanges16Day_PropVals)

mcpRangesAnnual_Path = mcpFolder + "\\MCPAnnualRanges.gdb"
mcpRangesAnnual_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
mcpRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
mcpRangesAnnual_PropVals.Add(System.Tuple[str,str]("DATABASE", mcpRangesAnnual_Path))
mcpRangesAnnual_Props = GeodatabaseHelper.CreateConnection(mcpRangesAnnual_PropVals)

In [ ]:
#Define Temporality databases
tempmetrics16Day_Path = tempmetricsFolder + "\\TempMetrics16Day.gdb"
tempmetrics16Day_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
tempmetrics16Day_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
tempmetrics16Day_PropVals.Add(System.Tuple[str,str]("DATABASE", tempmetrics16Day_Path))
tempmetrics16Day_Props = GeodatabaseHelper.CreateConnection(tempmetrics16Day_PropVals)

tempmetricsAnnual_Path = tempmetricsFolder + "\\TempMetricsAnnual.gdb"
tempmetricsAnnual_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
tempmetricsAnnual_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
tempmetricsAnnual_PropVals.Add(System.Tuple[str,str]("DATABASE", tempmetricsAnnual_Path))
tempmetricsAnnual_Props = GeodatabaseHelper.CreateConnection(tempmetricsAnnual_PropVals)

In [ ]:
# Define the TrajPathMetrics database
trajPathMetrics16Day_Path = trajPathMetricsFolder + "\\TrajPathMetrics16Day.gdb"
trajPathMetrics16Day_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
trajPathMetrics16Day_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
trajPathMetrics16Day_PropVals.Add(System.Tuple[str,str]("DATABASE", trajPathMetrics16Day_Path))
trajPathMetrics16Day_Props = GeodatabaseHelper.CreateConnection(trajPathMetrics16Day_PropVals)

trajPathMetricsAnnual_Path = trajPathMetricsFolder + "\\TrajPathMetricsAnnual.gdb"
trajPathMetricsAnnual_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
trajPathMetricsAnnual_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
trajPathMetricsAnnual_PropVals.Add(System.Tuple[str,str]("DATABASE", trajPathMetricsAnnual_Path))
trajPathMetricsAnnual_Props = GeodatabaseHelper.CreateConnection(trajPathMetricsAnnual_PropVals)

In [ ]:
#Define database for the WDPA feature data
wdpaGDB_Path = SpatialDataFolder + "\\WDPA_August2013.gdb"
wdpaGDB_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
wdpaGDB_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
wdpaGDB_PropVals.Add(System.Tuple[str,str]("DATABASE", wdpaGDB_Path))
wdpaGDB_Props = GeodatabaseHelper.CreateConnection(wdpaGDB_PropVals)

In [ ]:
#Define the Regions database
regionsGDB_Path = SpatialDataFolder + "\\Regions.gdb"
regionsGDB_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
regionsGDB_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
regionsGDB_PropVals.Add(System.Tuple[str,str]("DATABASE", regionsGDB_Path))
regionsGDB_Props = GeodatabaseHelper.CreateConnection(regionsGDB_PropVals)

In [ ]:
#Define the Spatial database
spatialGDB_Path = SpatialDataFolder + "\\SpatialData.gdb"
spatialGDB_PropVals = System.Collections.Generic.List[System.Tuple[str,str]]()
spatialGDB_PropVals.Add(System.Tuple[str,str]("DATABASETYPE", "FGDB"))
spatialGDB_PropVals.Add(System.Tuple[str,str]("DATABASE", spatialGDB_Path))
spatialGDB_Props = GeodatabaseHelper.CreateConnection(spatialGDB_PropVals)

# AnimalTracking Database, Regions & TrackingMaster tables

In [ ]:
DBI = TrackingDatabaseESRIAdapter("DB_CONNECTION_PROPERTIES=ATDB_Connection,DATABASETYPE=FGDB,DATABASE=" + animalTrackingDB_Path)

### Join TM & Regions tables

In [ ]:
#1. Manually export the regions table to a csv from the server Postgres database
#   COPY Regions TO 'E:/Research/2015-04-01 PanAfEl Manuscript/Spatial Data/Regions.csv' CSV HEADER;
#2. Manually create a Regions.gdb database in the 'Spatial Data' folder
#3. Import regions csv file to a table called 'Regions' to the Regions.gdb database:

arcpy.TableToTable_conversion(in_rows=SpatialDataFolder + "\\Regions.csv",
                              out_path=SpatialGDB_Path + "\\Regions.gdb", out_name="Regions")

#4. Manually create a 'MetaRegions' feature class in the Regions.gdb database with polygons for each meta-region:
#   Central,South,East,West
#5. Add a 'MetaRegion' column to the Regions feature class and populate with appropriate metaregion designation

In [ ]:
#Join the TrackingMaster table with the Regions table (only need to do this once)
#arcpy.JoinField_management(animalTrackingDB_Path + "\\TRACKINGMASTER", "Chronofile", 
#                           regionsGDB_Path + "\\Regions","Chronofile",["Region","Country","MetaRegion"])

# Movement Data

### Extract Data from AnimalTracking Database

In [ ]:
#Select both species from the db
elesQuery = "SPECIES='Elephant' OR SPECIES='Forest Elephant'"

#Open the FeatureClass that will define the Metaregions
regionFC = FeatureDataHelper.ReturnFeatureClass2(regionsGDB_Props,"MetaRegionsDissolve")
animalInfoList = DBI.GetAnimalInfoList(elesQuery,regionFC) #Use a regional based spatial query

In [ ]:
#Define a TrajectoryFilter
minTime = System.TimeSpan.FromMinutes(2.0) #The minimum time needed between points otherwise the later point gets filtered
trajFilt = TrajectoryFilter(0.5,minTime,8.0,startDate,endDate) #MinDist_Meters, MinTime_Minutes, MaxSpeed_Km/Hr, Start, End

#Define the DatasetExtractionParameters object
DEP = DatasetExtractionParameters()
DEP.AnimalTrackingDB = DBI
DEP.MergeType = MergeOption.ByAnimal #Because data needs to be in UTM then had to do separate feature classes and not merge them
DEP.startTimeOption = StartTimeOption.MostRecent
DEP.endTimeOption = EndTimeOption.LessRecent
DEP.startTime = startDate
DEP.endTime = endDate
DEP.PointFilter = trajFilt #filter the data

#Define the DatasetStorageParameters object
DSP = ESRIDatasetStorage()
DSP.LyrFilePath = MovDataFolder
DSP.OutputUTM = True #Will calculating areas so need projected coordinates
DSP.CreateTracks = False 
DSP.OutputDBProps = MovGDB_Props

In [ ]:
#Perform the extraction
MDE = MovementDataExtractor()
MDE.ExtractDatasets(animalInfoList, DEP, DSP);

### Clean Data

In [ ]:
#Delete entire feature classes that shouldn't be included in analysis
arcpy.Delete_management (MovGDB_Path + "\\Wangari_Maathai_Locs") #Data reflected at the equator and so can't tell what's N vs S
arcpy.Delete_management (MovGDB_Path + "\\FortyFive_Locs") #Single animal from Botswana
arcpy.Delete_management (MovGDB_Path + "\\Kiramatian_Locs")  #ACC elephant, not STE
arcpy.Delete_management (MovGDB_Path + "\\Lorna_Locs")  #ACC elephant, not STE
arcpy.Delete_management (MovGDB_Path + "\\Dionysus_Locs")  #Zim elephant, not STE
arcpy.Delete_management (MovGDB_Path + "\\Thor_Locs")  #Zim elephant, not STE
arcpy.Delete_management (MovGDB_Path + "\\Zeus_Locs")  #Zim elephant, not STE

In [ ]:
#Define a function to delete individual datapoints
def delFeatures(cleanName, sqlexpression):
    tempLayer = "movSelect"
    arcpy.MakeFeatureLayer_management(MovGDB_Path + "\\" + cleanName, tempLayer)
    arcpy.SelectLayerByAttribute_management(tempLayer, "NEW_SELECTION", sqlexpression)
    if int(arcpy.GetCount_management(tempLayer).getOutput(0)) > 0:
        delCount = arcpy.DeleteFeatures_management(tempLayer)
    arcpy.Delete_management( tempLayer)

In [ ]:
#This animal was physically confined within a fence for a period of time at start of tracking dataset
#Delete the records before elephant was free ranging
name = 'Ngelesha'
sqlexpression = "MovDataID='" + name + "' AND fixtime < date'2008-11-08 10:00:26'"
cleanName = FeatureDataHelper.CleanupName(name) + '_locs'
delFeatures(cleanName,sqlexpression)

In [ ]:
#Individual Point deletions
#Manually identify points for deletion from each animal's dataset that weren't picked up by the filter
#Requires examining each dataset for obvious errors and listing them in csv file
pointsToDel = pd.read_csv(MovDataFolder + "\\Filter_Indiv_Points.csv",header=0)

In [ ]:
for i in range(0,len(pointsToDel.index)):
    name = pointsToDel['Name'][i]
    sqlexpression = "MovDataID='" + name + "' AND fixtime = date'" + pointsToDel['Fixtime'][i] + "'"
    cleanName = FeatureDataHelper.CleanupName(name) + '_locs'
    try:
        delFeatures(cleanName,sqlexpression)
    except:
        pass

### Ele Names Array

In [ ]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

In [ ]:
arcpy.env.workspace=MovGDB_Path
fcs = arcpy.ListFeatureClasses("*_Locs")
fcCount=0
countFeatures=0
eleNamesArray=[]
for fc in fcs:
    thisCount = int(arcpy.GetCount_management(fc).getOutput(0))
    countFeatures += thisCount
    if thisCount > 0:
        fcCount+=1
        eleNamesArray.append(str(unique_values(fc,'MovDataID')[0])) #Grab the first unique value (should just be one)
    
del(fcs)
print fcCount, countFeatures
eleNamesArray = sorted(eleNamesArray)
print eleNamesArray

### Project & Merge Movement Datasets

In [ ]:
# It can be useful, in some cases, to have a single feature class with all movement data

In [ ]:
arcpy.env.workspace=MovGDB_Path
arcpy.env.Overwrite=True
fcs = arcpy.ListFeatureClasses("*_Locs")
for fc in fcs:
    arcpy.Project_management(in_dataset=fc, out_dataset=fc + '_gcs', out_coor_system=wgs84GCS)

In [ ]:
#Merge all projected movement feature classes
arcpy.env.workspace=MovGDB_Path
arcpy.env.Overwrite=True
fcs = arcpy.ListFeatureClasses("*_gcs")
mergeGCSDatasets=[]
for fc in fcs:
    mergeGCSDatasets.append(fc)
    
arcpy.Merge_management(mergeGCSDatasets, MovGDB_Path + "\\MergeAllLocsGCS")

# Moving Window Definitions

In [ ]:
window = MovingWindow()

In [ ]:
mcp = MCPRange()
def CalcMCPRange(traj,calcParams):
    try:
        theresult = mcp.CalculateMCPRange(traj, calcParams)
        theresult.WriteCalcSummaryTable('MCPSummaryTable', calcParams.dbProps)
        del(theresult)
    except Exception as e:
        print(e)
        
#Define the function delegate
mcpRangeFuncDelegate = MovingWindow.MethodToIterate(CalcMCPRange)

In [ ]:
etd = EllipticalTimeDensityRange()
def CalcETDRange(traj, calcParams): 
    #Define the ETD function that will run for each moving window timespan
    try:
        result = etd.CalculateETDRange(traj, calcParams)
        result.WriteCalcSummaryTable('ETDSummaryTable', calcParams.dbProps)
        del result
    except Exception as e:
        print(e)
    
#Define the function delegate
etdRangeFuncDelegate = MovingWindow.MethodToIterate(CalcETDRange)

In [ ]:
kde = KernelDensityRange()
def CalcKDERange(traj, calcParams): 
    #Define the ETD function that will run for each moving window timespan
    try:
        result = kde.CalculateKernelDensityRange(traj, calcParams)
        result.WriteCalcSummaryTable('KDESummaryTable', calcParams.dbProps)
        del result
    except Exception as e:
        print(e)
    
#Define the function delegate
kdeRangeFuncDelegate = MovingWindow.MethodToIterate(CalcKDERange)

In [ ]:
tempMet = TemporalMetrics()
def CalcTempMetrics(traj,calcParams):
    try:
        tempMet.CalculateTemporalityMetrics(traj, calcParams)
    except Exception as e:
        print(e)
        
tempMetFuncDelegate = MovingWindow.MethodToIterate(CalcTempMetrics)

In [ ]:
# Calculate Trajectory Path Metrics
trajPathMetrics = TrajectoryPathMetrics()
def trajPathMetricsFunc(traj,calcParams):
    theresult = trajPathMetrics.CalculateTrajectoryPathMetrics(traj, calcParams)
    #theresult.WriteCalcSummaryTable('TrajPathMetricsCalcResultsTable', TrajPathMetricsGDB_Props)
    del(theresult)
trajPathMetricsDelegate = MovingWindow.MethodToIterate(trajPathMetricsFunc)

# MCP Ranges

### Calculate MCP Ranges

In [ ]:
#Calculate 16-Day & yearly MCP Ranges for all elephants
t1=dt.datetime.now()

for name in eleNamesArray:
    print("Analysing: " + name)
    cleanName = FeatureDataHelper.CleanupName(name) + '_Locs'
    
    #Open the movement data feature class
    MovFC = FeatureDataHelper.ReturnFeatureClass2(MovGDB_Props,cleanName)

    #Create a trajectory
    path = TrajectoryEngine.CreateTrajectoryFromPointFeatureClass(MovFC,'',"Fixtime",True)

    #Define the CalcID
    calcID = cleanName 

    #Calculate 16-Day Windows
    modisDateObject = MODIS16DayWindowUnit(path.StartTime)
    windowDates16Day = modisDateObject.DefineWindowTimes(path.StartTime,path.EndTime)
    
    #Calculate the MCP 16-Day Ranges
    mcpParams16Day = MCPRangeCalcParameters(percentiles,cleanName,True,name,calcID,
                                            path.StartTime,path.EndTime,mcpRanges16Day_Props)
    #Run the 16-day calculation
    window.IterateWindow(path,windowDates16Day,False,True,mcpRangeFuncDelegate,mcpParams16Day)
    
    #Calculate Annual Windows
    annualWindowObject = YearWindowUnit(path.StartTime,1)
    windowDatesAnnual = annualWindowObject.DefineWindowTimes(path.StartTime,path.EndTime,1.0)
    
    #Calculate the MCP Annual Range
    mcpParamsAnnual = MCPRangeCalcParameters(percentiles,cleanName,True,name,calcID,
                                             path.StartTime,path.EndTime,mcpRangesAnnual_Props)
    
    #Run the Annual calculation
    window.IterateWindow(path, windowDatesAnnual,False,True,mcpRangeFuncDelegate,mcpParamsAnnual)
      
    #Cleanup
    del path,MovFC,modisDateObject, windowDates16Day,mcpParams16Day,annualWindowObject,windowDatesAnnual,mcpParamsAnnual
    
    gc.collect()

t2=dt.datetime.now()
deltaT=t2-t1
print('The MCP calculation took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #22.12 minutes

### Project & Merge MCP Ranges

In [ ]:
#Project all 16-Day mcp ranges to common GCS projection
arcpy.env.workspace=mcpRanges16Day_Path
arcpy.env.Overwrite=True
fcs = arcpy.ListFeatureClasses()
for fc in fcs:
    arcpy.Project_management(in_dataset=fc, out_dataset=fc + '_gcs', out_coor_system=wgs84GCS)
del(fcs)
print("Complete.")

In [ ]:
#Merge all 16-day GCS mcp ranges
arcpy.env.workspace=mcpRanges16Day_Path
fcs = arcpy.ListFeatureClasses("*_gcs")
mergeGCSDatasets=[]
for fc in fcs:
    mergeGCSDatasets.append(fc)  
arcpy.Merge_management(mergeGCSDatasets, mcpRanges16Day_Path + "\\MCP16DayPolygonsGCS")
del(fcs)
print("Complete.")

In [ ]:
#Project all annual mcp ranges to common GCS projection
arcpy.env.workspace=mcpRangesAnnual_Path
arcpy.env.Overwrite=True
fcs = arcpy.ListFeatureClasses()
for fc in fcs:
    arcpy.Project_management(in_dataset=fc, out_dataset=fc + '_gcs', out_coor_system=wgs84GCS)
del(fcs)
print("Complete.")

In [ ]:
#Merge all annual GCS mcp ranges
arcpy.env.workspace=mcpRangesAnnual_Path
arcpy.env.Overwrite=True
fcs = arcpy.ListFeatureClasses("*_gcs")
mergeGCSDatasets=[]
for fc in fcs:
    mergeGCSDatasets.append(fc)
arcpy.Merge_management(mergeGCSDatasets, mcpRangesAnnual_Path + "\\MCPAnnualPolygonsGCS")
del(fcs)
print("Complete.")

In [ ]:
#Dissolve the merged yearly ranges
arcpy.env.workspace=mcpRangesAnnual_Path
arcpy.env.Overwrite=True
arcpy.Dissolve_management("MCPAnnualPolygonsGCS", "MCPAnnualPolygonsGCSDissolv")
print("Complete.")

In [ ]:
#Project the dissolved MCP polygons to Africa Albers Equal Area Conic projection to get the area in Km2
arcpy.env.workspace=mcpRangesAnnual_Path
arcpy.env.Overwrite=True
arcpy.Project_management(in_dataset="MCPAnnualPolygonsGCSDissolv", 
                         out_dataset="MCPAnnualPolygonsAlbersDissolv", 
                         out_coor_system=arcpy.SpatialReference(102022))
print("Complete.")

# KDE Ranges

In [ ]:
#Use this function if you need to re-run the KDE/ETD calculation for some reason 
#(e.g., the first time through there was an error with some grids
#and you want to run it again). This function will check the results table and 
#if a calc with the given start/end dates exists then
#it will remove those dates from the input windowDates array

# first delete any row entries that have an error
def alreadyRun(MovDataID, resultsDBPath, resultsTableName, windowDates):
    resultsTable = resultsDBPath + '\\' + resultsTableName 
    whereClause = "MovDataID='" + MovDataID + "'"
    cursor = arcpy.SearchCursor(dataset=resultsTable, where_clause=whereClause)
    for row in cursor:
        
        #remove the current dates from the windowDates list for any previously successful run
        curStart = row.getValue("StartDate")
        curEnd = row.getValue("EndDate")
        
        #Create a .Net Tuple
        curWin = System.Tuple[System.DateTime,System.DateTime](System.DateTime.Parse(curStart.strftime("%Y-%m-%d %H:%M:%S")),
                                                               System.DateTime.Parse(curEnd.strftime("%Y-%m-%d %H:%M:%S")))
        
        #print 'Removing' + curWin.ToString()
        windowDates.Remove(curWin)
    del cursor
    return windowDates

In [ ]:
#Calculate KDE Ranges for all elephants
t1=dt.datetime.now()

for name in eleNamesArray:
    print("Analysing: " + name)
    cleanName = FeatureDataHelper.CleanupName(name) + '_Locs'
    
    #Open the movement data feature class
    MovFC = FeatureDataHelper.ReturnFeatureClass2(MovGDB_Props,cleanName)

    #Create a trajectory
    path = TrajectoryEngine.CreateTrajectoryFromPointFeatureClass(MovFC,'',"Fixtime",True)

    #Define the CalcID
    calcID = cleanName

    #Calculate 16-Day Windows
    modisDateObject = MODIS16DayWindowUnit(path.StartTime)
    windowDates16Day = modisDateObject.DefineWindowTimes(path.StartTime,path.EndTime)

    #Calculate Annual Windows
    annualWindowObject = YearWindowUnit(path.StartTime,1)
    windowDatesAnnual = annualWindowObject.DefineWindowTimes(path.StartTime,path.EndTime,1.0)

    #Skip dates where the calc was already run
    windowDates16Day = alreadyRun(name, kdeRanges16Day_Path,'KDESummaryTable', windowDates16Day)
    windowDatesAnnual = alreadyRun(name, kdeRangesAnnual_Path,'KDESummaryTable', windowDatesAnnual)

    #Calculate the KDE 16-Day Range 
    if windowDates16Day.Count > 0:
        
        kdeParams16Day = KernelDensityRangeCalcParameters( 0., # smoothingParameter
                                                          'h_Ref', # smoothingMethod
                                                           6, # MaxSD
                                                           1E-20, # ProbCutOffVal
                                                           cleanName + '_etd', # outRasterName
                                                           1.3, # rasterExpansionRatio
                                                           200., # rasterResolution
                                                           name, # movDataID
                                                           calcID, # calcID
                                                           path.StartTime, # start
                                                           path.EndTime, # end
                                                           kdeRanges16Day_Props) # dbProps
        
        #Run the calculation
        window.IterateWindow(path, 
                             windowDates16Day,
                             False,
                             True,
                             kdeRangeFuncDelegate,
                             kdeParams16Day)
        
        #Cleanup
        del kdeParams16Day

    #Calculate the KDE Annual Range
    if windowDatesAnnual.Count > 0:
        
        kdeParamsAnnual = KernelDensityRangeCalcParameters(0.,
                                                          'h_Ref',
                                                           6,
                                                           1E-20,
                                                           cleanName + '_etd',
                                                           1.3,
                                                           200.,
                                                           name,
                                                           calcID,
                                                           path.StartTime,
                                                           path.EndTime,
                                                           kdeRangesAnnual_Props)
         
        
        #Run the calculation
        window.IterateWindow(path,
                             windowDatesAnnual,
                             False,
                             True,
                             kdeRangeFuncDelegate,
                             kdeParamsAnnual)
        
        #Cleanup
        del kdeParamsAnnual

    #Cleanup
    del path, modisDateObject, annualWindowObject, windowDates16Day, windowDatesAnnual

t2=dt.datetime.now()
deltaT=t2-t1
print('The KDE calculation took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #2810.91 minutes

### Clean up 16-Day KDE database

In [ ]:
#Figure out the names of all non-null rasters from the ETD Summary Table
resultsTable = kdeRanges16Day_Path + '\\' + 'KDESummaryTable'
nonNullKDERasters = []
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        nonNullKDERasters.append(ResultRaster)
del cursor
print str(len(nonNullKDERasters))

#Determine any rasters that are not part of the non-null list
arcpy.env.workspace=kdeRanges16Day_Path
rastToDel = []
rastCursor = arcpy.ListRasters()
for rast in rastCursor:
    if str(rast) not in nonNullKDERasters:
        rastToDel.append(str(rast))
del rastCursor
print str(len(rastToDel))

#delete these orphaned rasters from the database to clean it up
count=0
for delRast in rastToDel:
    count=count+1
    print str(count)
    arcpy.Delete_management(delRast)

### Clean up Annual KDE database

In [ ]:
#Figure out the names of all non-null rasters from the ETD Summary Table
resultsTable = kdeRangesAnnual_Path + '\\' + 'KDESummaryTable'
nonNullKDERasters = []
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        nonNullKDERasters.append(ResultRaster)
del cursor
print str(len(nonNullKDERasters))

#Determine any rasters that are not part of the non-null list
arcpy.env.workspace=kdeRangesAnnual_Path
rastToDel = []
rastCursor = arcpy.ListRasters()
for rast in rastCursor:
    if str(rast) not in nonNullKDERasters:
        rastToDel.append(str(rast))
del rastCursor
print str(len(rastToDel))

#delete these orphaned rasters from the database to clean it up
count=0
for delRast in rastToDel:
    count=count+1
    print str(count)
    arcpy.Delete_management(delRast)

### Calculate 16-Day KDE Percentile Polygons

In [ ]:
# Open the 16-Day KDE Summary Table and loop through the values
percentArea = PercentileArea()
resultsTable = kdeRanges16Day_Path + '\\' + 'KDESummaryTable'
gcsSR = FeatureDataHelper.CreateGeographicSR(4326)
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        print ResultRaster
        percentAreaParams = PercentileAreaCalculationParameters(percentiles, 1E-20, ResultRaster, kdeRanges16Day_Props,
                                                 "KDE16DayPolygonsGCS", True, gcsSR, row.getValue("MovDataID"),
                                                                row.getValue("CalcID"), kdeRanges16Day_Props)
        percentArea.CalculatePercentileAreas(percentAreaParams)
del cursor

### Calculate Annual KDE Percentile Polygons

In [ ]:
# Open the Annual KDE Summary Table and loop through the values
percentArea = PercentileArea()
resultsTable = kdeRangesAnnual_Path + '\\' + 'KDESummaryTable'
gcsSR = FeatureDataHelper.CreateGeographicSR(4326)
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        print ResultRaster
        percentAreaParams = PercentileAreaCalculationParameters(percentiles, 1E-20, ResultRaster, kdeRangesAnnual_Props,
                                                 "KDEAnnualPolygonsGCS", True, gcsSR, row.getValue("MovDataID"),
                                                                row.getValue("CalcID"), kdeRangesAnnual_Props)
        percentArea.CalculatePercentileAreas(percentAreaParams)
del cursor

#  ETD Ranges

In [ ]:
#Calculate ETD Ranges and Temporality Metrics for all elephants
t1=datetime.datetime.now()

for name in eleNamesArray:
    print("Analysing: " + name)
    cleanName = FeatureDataHelper.CleanupName(name) + '_Locs'
    
    #Open the movement data feature class
    MovFC = FeatureDataHelper.ReturnFeatureClass2(MovGDB_Props,cleanName)

    #Create a trajectory
    path = TrajectoryEngine.CreateTrajectoryFromPointFeatureClass(MovFC,'',"Fixtime",True)

    #Define the CalcID
    calcID = cleanName

    #Calculate 16-Day Windows
    modisDateObject = MODIS16DayWindowUnit(path.StartTime)
    windowDates16Day = modisDateObject.DefineWindowTimes(path.StartTime,path.EndTime)

    #Calculate Annual Windows
    annualWindowObject = YearWindowUnit(path.StartTime,1)
    windowDatesAnnual = annualWindowObject.DefineWindowTimes(path.StartTime,path.EndTime,1.0)

    #Skip dates where the calc was already run
    windowDates16Day = alreadyRun(name,etdRanges16Day_Path,'ETDSummaryTable', windowDates16Day)
    windowDatesAnnual = alreadyRun(name,etdRangesAnnual_Path,'ETDSummaryTable', windowDatesAnnual)

    #Calculate the ETD 16-Day Range 
    if windowDates16Day.Count > 0:
              
        #Provide default values so that parametrization will take place automatically for each window segment
        etdWeibullParams = EllipticalTimeDensityKernel2Parameters(1.0,1.0)
        
        #MaxDataGapSeconds,maxSpeed,maxSpeedPercent,intStepKmHr,WeibullParams,ProbCutOffVal,outRasterName,rasterExpansionRatio,rasterResolution,movDataID
        etdParams16Day = EllipticalTimeDensityRangeCalcParameters(43200,0.0,0.90,0.0001,etdWeibullParams,1E-20,cleanName + '_etd',
                                                                  1.3,200,name,calcID,path.StartTime,path.EndTime,etdRanges16Day_Props)
        #Run the calculation
        window.IterateWindow(path,windowDates16Day,False,True,etdRangeFuncDelegate,etdParams16Day)
        
        #Cleanup
        del etdParams16Day,etdWeibullParams

    #Calculate the ETD Annual Range
    if windowDatesAnnual.Count > 0:
        
        #Provide default values so that parametrization will take place automatically for each window segment
        etdWeibullParams = EllipticalTimeDensityKernel2Parameters(1.0,1.0)
        
        #MaxDataGapSeconds,maxSpeed,maxSpeedPercent,intStepKmHr,WeibullParams,ProbCutOffVal,outRasterName,rasterExpansionRatio,rasterResolution,movDataID
        etdParamsAnnual = EllipticalTimeDensityRangeCalcParameters(43200,0.0,0.90,0.0001,etdWeibullParams,1E-20,cleanName + '_etd',
                                                                   1.3,200,name,calcID,path.StartTime,path.EndTime,etdRangesAnnual_Props)
        #Run the calculation
        window.IterateWindow(path,windowDatesAnnual,False,True,etdRangeFuncDelegate,etdParamsAnnual)
        
        #Cleanup
        del etdParamsAnnual,etdWeibullParams

    #Cleanup
    del path, modisDateObject, annualWindowObject, windowDates16Day, windowDatesAnnual

t2=datetime.datetime.now()
deltaT=t2-t1
print('The ETD calculation took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #2810.91 minutes

### Clean up 16-Day ETD database

In [ ]:
#Figure out the names of all non-null rasters from the ETD Summary Table
resultsTable = etdRanges16Day_Path + '\\' + 'ETDSummaryTable'
nonNullETDRasters = []
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        nonNullETDRasters.append(ResultRaster)
del cursor
print str(len(nonNullETDRasters))

#Determine any rasters that are not part of the non-null list
arcpy.env.workspace=etdRanges16Day_Path
rastToDel = []
rastCursor = arcpy.ListRasters()
for rast in rastCursor:
    if str(rast) not in nonNullETDRasters:
        rastToDel.append(str(rast))
del rastCursor
print str(len(rastToDel))

#delete these orphaned rasters from the database to clean it up
count=0
for delRast in rastToDel:
    count=count+1
    print str(count)
    arcpy.Delete_management(delRast)

### Clean up Annual ETD database

In [ ]:
#Figure out the names of all non-null rasters from the ETD Summary Table
resultsTable = etdRangesAnnual_Path + '\\' + 'ETDSummaryTable'
nonNullETDRasters = []
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        nonNullETDRasters.append(ResultRaster)
del cursor
print str(len(nonNullETDRasters))

#Determine any rasters that are not part of the non-null list
arcpy.env.workspace=etdRangesAnnual_Path
rastToDel = []
rastCursor = arcpy.ListRasters()
for rast in rastCursor:
    if str(rast) not in nonNullETDRasters:
        rastToDel.append(str(rast))
del rastCursor
print str(len(rastToDel))

#delete these orphaned rasters from the database to clean it up
count=0
for delRast in rastToDel:
    count=count+1
    print str(count)
    arcpy.Delete_management(delRast)

### Calculate 16-Day ETD Percentile Polygons

In [ ]:
#Open the 16-Day ETD Summary Table and loop through the values
percentArea = PercentileArea()
resultsTable = etdRanges16Day_Path + '\\' + 'ETDSummaryTable'
gcsSR = FeatureDataHelper.CreateGeographicSR(4326)
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        print ResultRaster
        percentAreaParams = PercentileAreaCalculationParameters(percentiles, 1E-20, ResultRaster, etdRanges16Day_Props,
                                                 "ETD16DayPolygonsGCS", True, gcsSR, row.getValue("MovDataID"), row.getValue("CalcID"),
                                                 etdRanges16Day_Props)
        percentArea.CalculatePercentileAreas(percentAreaParams)
del cursor

### Calculate Annual ETD Percentile Polygons

In [ ]:
#Open the Annual ETD Summary Table and loop through the values
percentArea = PercentileArea()
resultsTable = etdRangesAnnual_Path + '\\' + 'ETDSummaryTable'
gcsSR = FeatureDataHelper.CreateGeographicSR(4326)
cursor = arcpy.SearchCursor(dataset=resultsTable)
for row in cursor:
    ResultRaster = row.getValue("ResultRaster")
    if ResultRaster != "":
        print ResultRaster
        percentAreaParams = PercentileAreaCalculationParameters(percentiles, 1E-20, ResultRaster, etdRangesAnnual_Props,
                                                 "ETDAnnualPolygonsGCS", True, gcsSR, row.getValue("MovDataID"),
                                                                row.getValue("CalcID"), etdRangesAnnual_Props)
        percentArea.CalculatePercentileAreas(percentAreaParams)
del cursor

# Temporality Metrics

In [ ]:
#Calculate temporality metrics
t1=datetime.datetime.now()

for name in eleNamesArray:
    print("Analysing: " + name)
    cleanName = FeatureDataHelper.CleanupName(name) + '_Locs'
    
    #Open the movement data feature class
    MovFC = FeatureDataHelper.ReturnFeatureClass2(MovGDB_Props,cleanName)

    #Create a trajectory
    path = TrajectoryEngine.CreateTrajectoryFromPointFeatureClass(MovFC,'',"Fixtime",True)

    #Define the CalcID
    calcID = cleanName 
    
    #Calculate Modis 16-Day windows
    modisDateObject = MODIS16DayWindowUnit(path.StartTime)
    windowDates16Day = modisDateObject.DefineWindowTimes(path.StartTime,path.EndTime)

    tempMetParams16Day = TemporalMetricsCalcParameters("TemporalityMetrics",True,name,calcID,
                                                       path.StartTime,path.EndTime,tempmetrics16Day_Props) 
    
    #Run the 16-Day calculation
    window.IterateWindow(path, windowDates16Day,False,True,tempMetFuncDelegate,tempMetParams16Day)
    
    #Calculate Annual Windows
    annualWindowObject = YearWindowUnit(path.StartTime,1)
    windowDatesAnnual = annualWindowObject.DefineWindowTimes(path.StartTime,path.EndTime,1.0)

    tempMetParamsAnnual = TemporalMetricsCalcParameters("TemporalityMetrics",True,name,calcID,
                                                        path.StartTime,path.EndTime,tempmetricsAnnual_Props) 
    #Run the Annual calculation
    window.IterateWindow(path, windowDatesAnnual,False,True,tempMetFuncDelegate,tempMetParamsAnnual)
    
    #Cleanup
    del path, MovFC
    del tempMetParams16Day, modisDateObject, windowDates16Day
    del tempMetParamsAnnual, annualWindowObject, windowDatesAnnual
    gc.collect()

t2=datetime.datetime.now()
deltaT=t2-t1
print('The Temporality Metrics calculation took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #7.06 minutes

## Trajectory Path Metrics

In [ ]:
for name in eleNamesArray:
    print("Analysing: " + name)
    
    cleanName = FeatureDataHelper.CleanupName(name) + '_Locs'
    
    #Open the movement data feature class
    MovFC = FeatureDataHelper.ReturnFeatureClass2(MovGDB_Props,cleanName)

    #Create a trajectory
    path = TrajectoryEngine.CreateTrajectoryFromPointFeatureClass(MovFC,'',"Fixtime",True)

    #Define the CalcID
    calcID = cleanName 
    
    #Calculate Modis 16-Day windows
    modisDateObject = MODIS16DayWindowUnit(path.StartTime)
    windowDates16Day = modisDateObject.DefineWindowTimes(path.StartTime, path.EndTime)
    
    
    Params16Day = TrajectoryPathMetricsCalcParameters("TrajPathMetrics", True,  name, calcID,
                                                 path.StartTime, path.EndTime, trajPathMetrics16Day_Props) 
    
    #Run the 16-Day calculation
    window.IterateWindow(path, windowDates16Day,False,True,trajPathMetricsDelegate,Params16Day)
    
    #Calculate Annual Windows
    annualWindowObject = YearWindowUnit(path.StartTime,1)
    windowDatesAnnual = annualWindowObject.DefineWindowTimes(path.StartTime,path.EndTime,1.0)

    ParamsAnnual = TrajectoryPathMetricsCalcParameters("TrajPathMetrics", True,  name, calcID,
                                                 path.StartTime, path.EndTime, trajPathMetricsAnnual_Props)
    #Run the Annual calculation
    window.IterateWindow(path, windowDatesAnnual,False,True,trajPathMetricsDelegate,ParamsAnnual)
    
    #Cleanup
    del path, MovFC
    del Params16Day, modisDateObject, windowDates16Day
    del ParamsAnnual, annualWindowObject, windowDatesAnnual
    gc.collect()

# Join ETD Range Polygons with TrackingMaster, Temporality Metrics, and Regions tables

In [ ]:
arcpy.env.workspace = etdRanges16Day_Path
arcpy.JoinField_management("ETD16DayPolygonsGCS", "CalcID",
                           "ETDSummaryTable","CalcID",["StartDate","EndDate","WeibullSpeedParams"])
arcpy.JoinField_management("ETD16DayPolygonsGCS", "MovDataID",
                           tmTable,"Name",["Chronofile","Species","Sex","Region","Country","MetaRegion"])
arcpy.JoinField_management("ETD16DayPolygonsGCS", "CalcID",
                           tempmetrics16Day_Path + "\\TemporalityMetrics","CalcID",["NumPoints","TrajTimeHrs","TGI"])

In [ ]:
arcpy.env.workspace = etdRangesAnnual_Path
arcpy.JoinField_management("ETDAnnualPolygonsGCS", "CalcID",
                           "ETDSummaryTable","CalcID",["StartDate","EndDate","WeibullSpeedParams"])
arcpy.JoinField_management("ETDAnnualPolygonsGCS", "MovDataID",
                           tmTable,"Name",["Chronofile","Species","Sex","Region","Country","MetaRegion"])
arcpy.JoinField_management("ETDAnnualPolygonsGCS", "CalcID",
                           tempmetricsAnnual_Path + "\\TemporalityMetrics","CalcID",["NumPoints","TrajTimeHrs","TGI"])

# Protected Area Intersection

In [ ]:
spInt = SpatialIntegrator()

### Calculate the intersection area of each ETD 16-day range with the protected areas layer

In [ ]:
#16-Day
t1=datetime.datetime.now()
arcpy.env.workspace = etdRanges16Day_Path
spIntCalcParams = SpatialIntegratorFeatureCalcParams("WDPApoint_August2013_Clip_Dissolv",wdpaGDB_Props,'',
                                                     'ETD16DayPolygonsGCS','IntersectArea','',etdRanges16Day_Props)
spInt.SpatiallyIntegrateByZone(spIntCalcParams)
t2=datetime.datetime.now()
deltaT=t2-t1
print('The 16-Day intersection took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #482.23 minutes

### Calculate the % intersection of each ETD 16-day range with the protected areas layer

In [ ]:
#Add a field to hold the percent intersection value
arcpy.env.workspace = etdRanges16Day_Path
desc = arcpy.Describe('ETD16DayPolygonsGCS')
areafieldname = desc.AreaFieldName
arcpy.AddField_management(in_table='ETD16DayPolygonsGCS', field_name='IntersectPercent',
                          field_type='DOUBLE', field_is_nullable='NULLABLE')

#Calculate the percent intersect value
cursor = arcpy.UpdateCursor('ETD16DayPolygonsGCS')
for row in cursor:
    totalArea=row.getValue(areafieldname)
    intersectArea=row.getValue('IntersectArea')
    if totalArea > 0:
        row.setValue('IntersectPercent', intersectArea/totalArea)  
    else:
        row.setValue('IntersectPercent', 0 )
    cursor.updateRow(row)
del(cursor)

### Calculate the intersection area of each ETD annual range with the protected areas layer

In [ ]:
#Annual
t1=datetime.datetime.now()
arcpy.env.workspace = etdRangesAnnual_Path
spIntCalcParams = SpatialIntegratorFeatureCalcParams("WDPApoint_August2013_Clip_Dissolv",wdpaGDB_Props,'',
                                                     'ETDAnnualPolygonsGCS','IntersectArea','',etdRangesAnnual_Props)
spInt.SpatiallyIntegrateByZone(spIntCalcParams)
t2=datetime.datetime.now()
deltaT=t2-t1
print('The Annual intersection took: ' + str(deltaT.total_seconds()/60) + ' minutes.') #32.6 minutes

### Calculate the % intersection of each annual range with the protected areas layer

In [ ]:
#Add a field to hold the percent intersection value
arcpy.env.workspace = etdRangesAnnual_Path
desc = arcpy.Describe('ETDAnnualPolygonsGCS')
areafieldname = desc.AreaFieldName
arcpy.AddField_management(in_table='ETDAnnualPolygonsGCS',
                          field_name='IntersectPercent', field_type='DOUBLE', field_is_nullable='NULLABLE')

#Calculate the percent intersect value
cursor = arcpy.UpdateCursor('ETDAnnualPolygonsGCS')
for row in cursor:
    totalArea=row.getValue(areafieldname)
    intersectArea=row.getValue('IntersectArea')
    if totalArea > 0:
        row.setValue('IntersectPercent', intersectArea/totalArea)  
    else:
        row.setValue('IntersectPercent', 0 )
    cursor.updateRow(row)
del(cursor)

# Calculate Covariates for ETD Range Polygons

### Create Pandas dataframes from the output ETD polygons feature classes

In [ ]:
#Select the 90% percentile of the ETD model as per Wall et al. 2014
global_where_clause = 'DesiredPercentile=0.90 AND Area > 0.0' 

In [ ]:
def ConvertFeatureClassToPandasDataFrame(infc, where_clause):
    df = None
    try:
        convertObj=[]
        desc = arcpy.Describe(infc)
        
        #Shape field name
        shapefieldname=""
        try:
            shapefieldname = desc.ShapeFieldName 
        except:
            pass
        
        #OID field name
        OIDFieldName = desc.OIDFieldName 
        
        # all field names
        field_names = [field.name for field in arcpy.ListFields(infc)]
        field_names.remove(shapefieldname)

        for row in arcpy.SearchCursor(infc, where_clause, field_names):
            curRow=[]
            for field in field_names:
                if field != shapefieldname:
                    curRow.append(row.getValue(field))
#                 else:
#                     curRow.append(ConvertArcGISPolygonToEE_MultiPolygon(row.getValue(field)))
            convertObj.append(curRow)
        
        df = pd.DataFrame(convertObj)
        df.columns = field_names

    except Exception as e:
        s = str(e)
        print(s)
    finally:
        return df

In [ ]:
df16Day = ConvertFeatureClassToPandasDataFrame(etdRanges16Day_Path + "\\" + 'ETD16DayPolygonsGCS',
                                               where_clause = global_where_clause)
df16Day.index = df16Day.OBJECTID
print(len(df16Day.index)) #12467
df16Day.head()

In [ ]:
dfAnnual = ConvertFeatureClassToPandasDataFrame(etdRangesAnnual_Path + "\\" + 'ETDAnnualPolygonsGCS',
                                               where_clause = global_where_clause)
dfAnnual.index = dfAnnual.OBJECTID
print(len(dfAnnual.index)) #755
dfAnnual.head()

In [ ]:
# # Compare against old dataframe
# df16Day = pd.read_csv(filepath_or_buffer=etd16DayAnalysisTable,header=0)
# print(len(df16Day.index)) #49904
# df16Day = df16Day[df16Day.Area > 0.0]
# print(len(df16Day.index))
# df16Day = df16Day[df16Day.DesiredPercentile == 0.90]
# print(len(df16Day.index))
# df16Day.index = df16Day.OBJECTID
# df16Day.head()

### Define GEE extraction functions

In [ ]:
#Initialize the Earth Engine object, using your authentication credentials.
ee.Initialize()
print "GEE Initialized"

In [ ]:
#Define the stack reducer
imgReduce = ee.Reducer.mean()

#Define the region reducer
polyReduce = ee.Reducer.mean()

#Define the area reducer scale
reduceScale = 1000 #meters

In [ ]:
#16-Day
def Extract16Day(tempFeat):
    start = tempFeat.get('startDate')
    end = tempFeat.get('endDate')
    end1 = tempFeat.get('endDatePlusOne')

    #NDVI
    ndviReduce = ee.ImageCollection('MODIS/MCD43A4_NDVI'). \
    select('NDVI').filterDate(start,end1).reduce(imgReduce).\
    reduceRegion(polyReduce, tempFeat.geometry(),reduceScale)

    #NDWI
    ndwiReduce = ee.ImageCollection('MODIS/MCD43A4_NDWI').\
    select('NDWI').filterDate(start,end1).reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)

    #EVI
    eviReduce = ee.ImageCollection('MODIS/MCD43A4_EVI').\
    select('EVI').filterDate(start,end1).reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)

    #LST
    def calibrateLST(image):
        return image.multiply(ee.Image(0.02)).subtract(ee.Image(273.15)) # band has a 0.02 scale
    lstColl = ee.ImageCollection('MODIS/MOD11A2').filterDate(start,end).select('LST_Day_1km').map(calibrateLST)
    tempReduce = lstColl.reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
      
    #Tree
    treeReduce = ee.ImageCollection('MODIS/051/MOD44B').\
    select('Percent_Tree_Cover').reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)

    #TRMM - Note that we use all images in the stack since they are more frequent than the 16-Day
    trmmReduce = ee.ImageCollection('TRMM/3B42').\
    select(['precipitation']).filterDate(start,end).reduce(ee.Reducer.sum()).\
    reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)

    #Slope
    slopeReduce = ee.Algorithms.Terrain(ee.Image('USGS/SRTMGL1_003')).\
    select(['slope']).reduceRegion(polyReduce, tempFeat.geometry(), reduceScale)

    #HF
    hfImg = ee.Image('users/walljcg/hf_wcs_2009')
    hfMasked = hfImg.updateMask(hfImg.gte(0)) #Need to mask out NoData Values = -9999
    hfReduce = hfMasked.reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #Water
    waterReduce = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').\
    select('occurrence').unmask(0).reduceRegion(polyReduce, tempFeat.geometry(), reduceScale)

    return tempFeat.set({
        'ndviVal':ndviReduce,
        'ndwiVal':ndwiReduce,
        'eviVal':eviReduce,
        'tempVal':tempReduce, 
        'treeVal':treeReduce,
        'trmmVal':trmmReduce,
        'slopeVal':slopeReduce,
        'hfVal':hfReduce,
        'waterVal': waterReduce})

In [ ]:
#Annual
def ExtractAnnual(tempFeat):
    
    start = tempFeat.get('startDate')
    end = tempFeat.get('endDate')
    
    #NDVI
    ndviReduce = ee.ImageCollection('MODIS/MCD43A4_006_NDVI').\
    select('NDVI').filterDate(start,end).reduce(imgReduce).reduceRegion(polyReduce, tempFeat.geometry(),reduceScale)
    
    #NDWI
    ndwiReduce = ee.ImageCollection('MODIS/MCD43A4_006_NDWI').\
    select('NDWI').filterDate(start,end).reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #EVI
    eviReduce = ee.ImageCollection('MODIS/MCD43A4_006_EVI').\
    select('EVI').filterDate(start,end).reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #LST
    def calibrateLST(image):
        return image.multiply(ee.Image(0.02)).subtract(ee.Image(273.15)) # band has a 0.02 scale
    lstColl = ee.ImageCollection('MODIS/006/MOD11A2').filterDate(start,end).select('LST_Day_1km').map(calibrateLST)
    tempReduce = lstColl.reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #Tree
    treeReduce = ee.ImageCollection('MODIS/051/MOD44B').\
    select('Percent_Tree_Cover').reduce(imgReduce).reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #TRMM
    trmmReduce = ee.ImageCollection('TRMM/3B42').\
    select(['precipitation']).filterDate(start,end).reduce(ee.Reducer.sum()).\
    reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)
    
    #Slope
    slopeReduce = ee.Algorithms.Terrain(ee.Image('USGS/SRTMGL1_003')).\
    select(['slope']).reduceRegion(polyReduce, tempFeat.geometry(),reduceScale)
    
    #HF
    hfImg = ee.Image('users/walljcg/hf_wcs_2009')
    hfMasked = hfImg.updateMask(hfImg.gte(0)) #Need to mask out NoData Values = -9999
    hfReduce = hfMasked.reduceRegion(polyReduce,tempFeat.geometry(),reduceScale)

    #Water
    waterReduce = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').\
    select('occurrence').unmask(0).reduceRegion(polyReduce, tempFeat.geometry(), reduceScale)

    return tempFeat.set({'ndviVal':ndviReduce,
                         'ndwiVal':ndwiReduce,
                         'eviVal':eviReduce,
                         'tempVal':tempReduce, 
                         'treeVal':treeReduce,
                         'trmmVal':trmmReduce,
                         'slopeVal':slopeReduce,
                         'hfVal':hfReduce,
                         'waterVal': waterReduce})

In [ ]:
def ExtractCovariates(infc, extract_func, where_clause):
    results = []
    t1=dt.datetime.now()
    counter = 0

    desc = arcpy.Describe(infc)
    
    OIDFieldName = desc.OIDFieldName
    shapefieldname = desc.ShapeFieldName
    field_names = [OIDFieldName,shapefieldname]

    for row in arcpy.SearchCursor(infc, where_clause, field_names):
        try:
            curRow=dict()
            for name in field_names:
                if name != shapefieldname:
                    curRow[name]=row.getValue(name)
                else:
                    rings = ConvertArcGISPolygonToEE_MultiPolygon(row.getValue(name))
                    poly = ee.Geometry.MultiPolygon(rings)
                    GEEfeat = ee.Feature(poly,{'startDate':convertDateTimeToUnixTime(row.getValue('StartDate')),
                                       'endDate': convertDateTimeToUnixTime(row.getValue('EndDate')),
                                       'endDatePlusOne': convertDateTimeToUnixTime(row.getValue('StartDate') +
                                                                                   dt.timedelta(seconds=1))})
                    success = False
                    single_result = None
                    while success==False:
                        try:
                            single_result = ee.FeatureCollection(GEEfeat).map(extract_func).getInfo()
                            success = True
                        except Exception, e:
                            print str(e)
                            print('An ee error occurred...re-trying...')
                            time.sleep(5)  #Pause for 5 seconds before trying again

                    vals = single_result['features'][0]['properties']
                    curRow['ndviVal']=vals.get('ndviVal', np.nan).get('NDVI_mean', np.nan)
                    curRow['ndwiVal']=vals.get('ndwiVal', np.nan).get('NDWI_mean', np.nan)
                    curRow['eviVal']=vals.get('eviVal', np.nan).get('EVI_mean', np.nan)
                    curRow['tempVal']=vals.get('tempVal', np.nan).get('LST_Day_1km_mean', np.nan)
                    curRow['treeVal']=vals.get('treeVal', np.nan).get('Percent_Tree_Cover_mean', np.nan)
                    curRow['trmmVal']=vals.get('trmmVal', np.nan).get('precipitation_sum', np.nan)
                    curRow['hfVal']=vals.get('hfVal', np.nan).get('b1', np.nan)
                    curRow['slopeVal']=vals.get('slopeVal', np.nan).get('slope', np.nan)
                    curRow['waterVal']=vals.get('waterVal', np.nan).get('occurrence', np.nan)
            
            results.append(curRow)
            counter = counter + 1
            print(str(counter) + ': ' + str(curRow))
        except Exception as e:
            print str(e)
    t2=dt.datetime.now()
    deltaT=t2-t1
    print('The covariate extraction took: ' + str(deltaT.total_seconds()/60) + ' minutes.') # 263.6 minutes
    return results

In [ ]:
df16DayCovars = pd.DataFrame(ExtractCovariates(etdRanges16Day_Path + "\\" + 'ETD16DayPolygonsGCS',
                                               Extract16Day, global_where_clause))
df16DayCovars.index = df16DayCovars.OBJECTID
print(len(df16DayCovars.index))
df16DayCovars.head()

In [ ]:
# Merge the two dataframes
df16Day = df16Day.merge(df16DayCovars)
print(len(df16Day.index))
df16Day.head()

In [ ]:
df16Day.drop('SHAPE_Length',axis=1, inplace=True)
df16Day.drop('SHAPE_Area',axis=1, inplace=True)
df16Day.drop('SHAPE',axis=1, inplace=True)

In [ ]:
df16Day.to_csv(path_or_buf=etd16DayAnalysisTable + '_new',
             date_format='%Y-%m-%d %H:%M:%S', index=False, na_rep='NA')

### Extract Annual Covariate Info using GEE

In [ ]:
dfAnnualCovars = pd.DataFrame(ExtractCovariates(etdRangesAnnual_Path + "\\" + 'ETDAnnualPolygonsGCS',
                                               ExtractAnnual, global_where_clause))
dfAnnualCovars.index = dfAnnualCovars.OBJECTID
print(len(dfAnnualCovars.index))
dfAnnualCovars.head()

In [ ]:
# Merge the two dataframes
dfAnnual = dfAnnual.merge(dfAnnualCovars)
print(len(dfAnnual.index))

In [ ]:
dfAnnual.drop('SHAPE_Length',axis=1, inplace=True)
dfAnnual.drop('SHAPE_Area',axis=1, inplace=True)
dfAnnual.drop('SHAPE',axis=1, inplace=True)

In [ ]:
dfAnnual.to_csv(path_or_buf=etdAnnualAnalysisTable + '_new',
             date_format='%Y-%m-%d %H:%M:%S',index=False,na_rep='NA')

## Clean-up the 16-Day dataframe and re-export

In [ ]:
df16Day = pd.read_csv(filepath_or_buffer=etd16DayAnalysisTable +  '_new', header=0)
print(len(df16Day.index)) #12467

In [ ]:
#Define a custom filter to remove ranges for subject Taurus who was confined by a fence for different periods
def filterTaurusRanges(nm,testDates, t1, t2):
    result = False
    if nm=='Taurus':
        for i in testDates:
            if (i >= t1) & (i <= t2):
                result = True
    return result

taurusIntersectionDates = pd.read_csv(filepath_or_buffer=ignoreDataFile, header=0,
                                      usecols=['MovDataID','Fixtime'],parse_dates=['Fixtime'] )

In [ ]:
print df16Day.columns

In [ ]:
#temporarily change the water NA values to 0's
df16Day.waterVal.fillna(0, inplace=True)

In [ ]:
#Check the range of values
colsToCheck = ['ndviVal','ndwiVal','eviVal','tempVal','treeVal','trmmVal','hfVal','slopeVal', 'waterVal']
def valsRange(x):
    return [x.min(),x.max()]
for i in colsToCheck:
    print(i +': ' + str(valsRange(df16Day[i])))

In [ ]:
df16Day = df16Day.dropna() #Remove any rows that contain NA values
print(len(df16Day.index)) # 11980

#Convert to Sq.Km from m^2
df16Day["AreaKm"]=df16Day["Area"]/1000000

#Select the 90% percentile of the ETD model as per Wall et al. 2014
df16Day=df16Day[df16Day["DesiredPercentile"]==0.90]
print(len(df16Day.index)) # 11980

#Convert the start/end time columns to datetime objects and append to the dataframe
df16Day['StartDate2']=pd.to_datetime(df16Day['StartDate'])
df16Day['EndDate2']=pd.to_datetime(df16Day['EndDate'])

#Delete the Range areas known to be wrong 
df16Day['filterRange'] = False #Create a column with each value false
df16Day=df16Day[~((df16Day['MovDataID']=='Chukwi') &
          (df16Day['EndDate2'] > dt.datetime.strptime('2013-07-23 15:00:06',"%Y-%m-%d %H:%M:%S")))]
df16Day=df16Day[~((df16Day['MovDataID']=='Kuku') &
          (df16Day['EndDate2'] > dt.datetime.strptime('2013-08-24 14:22:23',"%Y-%m-%d %H:%M:%S")))]
df16Day=df16Day[~((df16Day['MovDataID']=='Winston') &
          (df16Day['EndDate2'] > dt.datetime.strptime('2003-12-19 03:00:56',"%Y-%m-%d %H:%M:%S")))]

#filter ranges that intersect times that Taurus was in the army camp
df16Day['filterRange'] = df16Day.apply(lambda x: 
                                      filterTaurusRanges(nm=x['MovDataID'],
                                      testDates = pd.to_datetime(taurusIntersectionDates['Fixtime']),
                                      t1 = pd.to_datetime(x['StartDate']),
                                      t2 = pd.to_datetime(x['EndDate'])),axis=1)
df16Day=df16Day[df16Day['filterRange']==False]

# drop the small home-range for the elephant Nuri where there was a mortality during the last two week period of the data
df16Day = df16Day.drop(df16Day[(df16Day['MovDataID']=='Nuri') & (df16Day['EndDate']>='2003-09-26 08:00:00')].index)

print(len(df16Day.index)) # 11,901

#Here we limit the data to anything with a TGI value equivalent to 12-hour sampling
#16 days = 16*24 = 384 hours
#There are m=(384 hours)/(12 hours)=32 12-hour segments in a 16-day period
#The annual TGI value for 12 hour sampling would be 1/m=1/32=0.03125

TGI_Cutoff=0.03125

#Filter ranges based on TGI scores
df16Day=df16Day[df16Day['TGI'] < TGI_Cutoff]
print(len(df16Day.index)) #10,319

df16Day.to_csv(path_or_buf=etd16DayAnalysisTableFilt + '_new', date_format='%Y-%m-%d %H:%M:%S', index=False)

### Clean-up the Annual dataframe and re-export

In [ ]:
dfAnnual = pd.read_csv(filepath_or_buffer=etdAnnualAnalysisTable + '_new', header=0)

In [ ]:
# temporarily change the water NA values to 0's
dfAnnual.waterVal.fillna(0, inplace=True)

In [ ]:
# Check the range of values
colsToCheck = ['ndviVal','ndwiVal','eviVal','tempVal','treeVal','trmmVal','hfVal','slopeVal', 'waterVal']
def valsRange(x):
    return [x.min(),x.max()]
for i in colsToCheck:
    print(i +': ' + str(valsRange(dfAnnual[i])))

In [ ]:
dfAnnual = dfAnnual.dropna() # Remove any rows that contain NA values
print(len(dfAnnual.index)) # 2968

#Convert to Sq.Km from m^2
dfAnnual["AreaKm"]=dfAnnual["Area"]/1000000

#Select the 90% percentile of the ETD model as per Wall et al. 2014
dfAnnual=dfAnnual[dfAnnual["DesiredPercentile"]==0.90]
print(len(dfAnnual.index)) # 742

#Convert the start/end time columns to datetime objects and append to the dataframe
dfAnnual['StartDate2']=pd.to_datetime(dfAnnual['StartDate'])
dfAnnual['EndDate2']=pd.to_datetime(dfAnnual['EndDate'])

#Delete the Range areas known to be wrong (note - might still delete the times that Taurus was in the army camp?)
dfAnnual=dfAnnual[~((dfAnnual['MovDataID']=='Chukwi') &
          (dfAnnual['EndDate2'] > dt.datetime.strptime('2013-07-23 15:00:06',"%Y-%m-%d %H:%M:%S")))]
dfAnnual=dfAnnual[~((dfAnnual['MovDataID']=='Kuku') &
          (dfAnnual['EndDate2'] > dt.datetime.strptime('2013-08-24 14:22:23',"%Y-%m-%d %H:%M:%S")))]


#filter ranges that intersect times that Taurus was in the army camp
dfAnnual['filterRange'] = dfAnnual.apply(lambda x: 
                                      filterTaurusRanges(nm=x['MovDataID'],
                                      testDates = pd.to_datetime(taurusIntersectionDates['Fixtime']),
                                      t1 = pd.to_datetime(x['StartDate']),
                                      t2 = pd.to_datetime(x['EndDate'])),axis=1)
dfAnnual=dfAnnual[dfAnnual['filterRange']==False]
print(len(dfAnnual.index)) # 738

#Here we limit the data to anything with a TGI value equivalent to 12-hour sampling
#There are m=(8760 hours)/(12 hours)=730 24-hour segments in a year
#There are m=(8760 hours)/(24 hours)=365 24-hour segments in a year
#There are m=(8760 hours)/(36 hours)=243.33 36-hour segments in a year
#There are m=(8760 hours)/(48 hours)=182.5 48-hour segments in a year
#The annual TGI value for 12 hour sampling would be 1/m=1/730=0.00136
#The annual TGI value for 24 hour sampling would be 1/m=1/365=0.0027
#The annual TGI value for 36 hour sampling would be 1/m=1/243.33=0.0041
#The annual TGI value for 48 hour sampling would be 1/m=1/182.5=0.00547

TGI_Cutoff=0.00136

#Filter ranges based on TGI scores
dfAnnual=dfAnnual[dfAnnual['TGI'] < TGI_Cutoff]
print(len(dfAnnual.index)) # 302

dfAnnual.to_csv(path_or_buf=etdAnnualAnalysisTableFilt + '_new', date_format='%Y-%m-%d %H:%M:%S', index=False)

# Dataset Summary

### Dataset Summary Table

In [ ]:
arcpy.env.workspace = MovGDB_Path

totalCount=0

datasetSummaryTable=[]

for fc in arcpy.ListFeatureClasses("*_Locs",feature_type="Point"):
    try:
        #Need to get the unique list of chronofiles for the given animal using the values in the CalcID column
        chronofiles = list(set([newrow[0] for newrow in arcpy.da.SearchCursor(fc,"CalcID",where_clause="")]))
        #print fc
        #print(chronofiles)
        for chrono in chronofiles:
            #Query the records for the given chronofile
            sqlString ="ORDER BY Fixtime ASC"
            fixtimes = [newrow[0] for newrow in arcpy.da.SearchCursor(fc,"Fixtime",
                                                                      where_clause="\"CalcID\"='" + chrono + "'",
                                                                      sql_clause=(None,sqlString))]
            
            #Record the number of fixes
            #print(str(len(fixtimes)))
            totalCount=totalCount+len(fixtimes)

            #Calculate the n-1 timespans from the fixtimes
            deltaTimes=[]
            for j in range(1,len(fixtimes)):
                deltaTime = fixtimes[j]-fixtimes[j-1]
                deltaTimes.append(deltaTime.total_seconds())

            #Determine the top 3 most frequent time-span lengths (in minutes)
            if len(deltaTimes) > 0: 
                #Bins: start at 2.5 mins then go in 5 minute increments upto 48 hours
                hist, bin_edges=np.histogram(deltaTimes,bins=range(150,172800,300))  
                histInd = np.argsort(hist,kind='mergesort')
                val1Min = bin_edges[histInd[len(histInd)-1]]
                val1Max = bin_edges[histInd[len(histInd)-1]+1]
                val2Min = bin_edges[histInd[len(histInd)-2]]
                val2Max = bin_edges[histInd[len(histInd)-2]+1]
                val3Min = bin_edges[histInd[len(histInd)-3]]
                val3Max = bin_edges[histInd[len(histInd)-3]+1]
                meanVal1=(val1Min+val1Max)/2
                meanVal2=(val2Min+val2Max)/2
                meanVal3=(val3Min+val3Max)/2

                #Get the first and last fix
                firstFix=fixtimes[0]
                lastFix=fixtimes[len(fixtimes)-1]

                #Calculate the Data Duration
                dataDuration=lastFix-firstFix
                dataDurationDays=dataDuration.total_seconds()/float(86400)

                #Determine what percentage of the data falls within the given bin
                count1=0
                count2=0
                count3=0
                for k in range(len(deltaTimes)):
                    if val1Max > deltaTimes[k] >= val1Min:
                        count1=count1+deltaTimes[k]
                    elif val2Max > deltaTimes[k] >= val2Min:
                        count2=count2+deltaTimes[k]
                    elif val3Max > deltaTimes[k] >= val3Min:
                        count3=count3+deltaTimes[k]         
                val1Percent = (count1/float(dataDuration.total_seconds()))*100
                if val1Percent < 1:#Set-Null if less than 1% of data
                    val1Percent=0
                    meanVal1=0 
                val2Percent = (count2/float(dataDuration.total_seconds()))*100
                if val2Percent < 1:#Set-Null if less than 1% of data
                    val2Percent=0
                    meanVal2=0
                val3Percent = (count3/float(dataDuration.total_seconds()))*100
                if val3Percent < 1:#Set-Null if less than 1% of data
                    val3Percent=0
                    meanVal3=0

                freqStats=[[meanVal1,val1Percent],[meanVal2,val2Percent],[meanVal3,val3Percent]]
                freqStats.sort(reverse=True, key=lambda tup: tup[1])

                #Setup the data_starts and data_stops output strings
                startString=""
                if not firstFix==None:
                    startString=firstFix.strftime("%Y-%m-%d %H:%M") #this gets the first valid datapoint
                endString=""
                if not lastFix==None:
                    endString=lastFix.strftime("%Y-%m-%d %H:%M") #this gets the last valid datapoint

                outRow=[str(fc),int(chrono),startString,dataDurationDays,freqStats[0][0]/60,len(fixtimes)]
                datasetSummaryTable.append(outRow)
    except:
        print('An error occurred...')

In [ ]:
#Join the calculated table above with the trackingmaster table
#Explicitly, set the correct datatypes for anything other than strings because the conversion from the ArcGIS types to Python types
#doesn't work very well and they end up as 'Objects' rather than floats, ints etc. and this leads to some weird results later

datasetSummaryDF=pd.DataFrame(datasetSummaryTable)
datasetSummaryDF.columns = ["FC","Chronofile","DataStarts","Duration_Days","SampFreqMode","DataPointCount"]
datasetSummaryDF['Chronofile'] = datasetSummaryDF['Chronofile'].astype(int)
datasetSummaryDF['DataStarts']=pd.to_datetime(datasetSummaryDF['DataStarts'])
#print datasetSummaryDF.head()

tmColumns=['Chronofile','Collar_type','Name','Species','Sex','Region','Country','MetaRegion']
tmDF = pd.DataFrame([row for row in arcpy.da.SearchCursor(tmTable, tmColumns)])
tmDF.columns = tmColumns
tmDF['Chronofile']=tmDF['Chronofile'].astype(int)
#print tmDF.head()

#Perform an inner join
datasetSummaryDF=pd.merge(datasetSummaryDF,tmDF,on='Chronofile',how='inner')

#Add a column with just a bunch of ones - useful for pivot table summaries
datasetSummaryDF['DS']=np.full((len(datasetSummaryDF.index),1),1,dtype=int)

#Display first few columns
print datasetSummaryDF.head()

#Print info about the dataframe
#datasetSummaryDF.info()

In [ ]:
#Export the datasetSummaryDF to a csv
datasetSummaryDF.to_csv(tablesFolder + '\\Dataset_Summary.csv', index=False, columns=datasetSummaryDF.columns[:-1])

In [ ]:
#Read in the datasetSummaryDF
datasetSummaryDF = pd.read_csv(tablesFolder + '\\Dataset_Summary.csv')

In [ ]:
datasetSummaryDF.columns

### Subject and dataset count

In [ ]:
#Number of unique individuals to start with 
print len(list(set(datasetSummaryDF['Name'])))

In [ ]:
#Number of datasets
print sum(datasetSummaryDF['DS'].values)

In [ ]:
#Number of individuals used in range analyses
print len(list(set(df16Day['MovDataID'])))
print len(list(set(dfAnnual['MovDataID'])))

In [ ]:
#Total datapoint count
print datasetSummaryDF['DataPointCount'].sum()

In [ ]:
# Median and IQR of individual positions
datasetSummaryDF.groupby('Name')['DataPointCount'].agg(np.sum).describe()
#IQR = 19173 - 2245 = 16928
#Median = 7417

#### Individual Count by Sex within Region, Country, Meta Region

In [ ]:
df1=datasetSummaryDF.pivot_table(index=['MetaRegion','Country','Region'],
                                 columns=['Sex'],
                                 values=['Name'],
                                 aggfunc=[lambda x: len(x.unique())],
                                 fill_value=0)

df1.fillna(value=0, inplace=True)
df1.to_csv(tablesFolder + '\\Subject_Count_by_Sex_within_Region.csv')

#### Datapoint Count by Sex within Region, Country, Meta Region

In [ ]:
df1=datasetSummaryDF.pivot_table(index=['MetaRegion','Country','Region'],
                                 columns=['DS'],
                                 values=['DataPointCount'],
                                 aggfunc=[sum],
                                 fill_value=0)

df1.fillna(value=0, inplace=True)
df1.to_csv(tablesFolder + '\\DataPoint_Count_by_Sex_within_Region.csv')

#### Collar Type Summary Table

In [ ]:
#Collar-Model-Summary Table in the Appendix
collarTypeSummaryDF=datasetSummaryDF.pivot_table(index=['Collar_type'], values=['Chronofile'],
                                                 aggfunc=lambda x: x.count(),fill_value=0,margins=False)
collarTypeSummaryDF.columns = ['Dataset Count']
collarTypeSummaryDF.to_csv(tablesFolder + '\\Collar-Tech-Summary.csv')
collarTypeSummaryDF

### Dataset Sampling Frequency Summary

In [ ]:
# What is the median dataset duration?
datasetSummaryDF['Duration_Days'].describe()

In [ ]:
# Number of individuals with over a year of data
dfgrp = datasetSummaryDF.groupby('Name')['Duration_Days'].agg(np.sum).to_frame()
dfgrp[dfgrp['Duration_Days']>=730]['Duration_Days'].describe()

In [ ]:
#How many datasets are there at each of the sampling frquencies
#And
#What is the median dataset duration for the different factor levels of SampFreq Mode?
#And
#What is the maximum dataset duration for the different factor levels of SampFreq Mode?

df1=datasetSummaryDF.pivot_table(index=['SampFreqMode'],values=['Duration_Days'],
                                 aggfunc=[lambda x:x.count(),np.median,np.max],fill_value=0)
df1.columns=['Dataset Count','Median Duration (Days)', 'Max Duration']
df1.to_csv(tablesFolder + '\\DataSet_Sampling_Summary.csv')
df1

### Collar Deployment Locations

In [ ]:
#Determine the first fix location from the supplied filtered & merged dataset

#Create a new blank feature class to hold the output
arcpy.env.workspace = spatialGDB_Path

collarDeploymentLocsFC = 'CollarDeploymentLocations'

if not arcpy.Exists(collarDeploymentLocsFC):
    arcpy.CreateFeatureclass_management(spatialGDB_Path, collarDeploymentLocsFC,
                                        "POINT", MovGDB_Path + "\\MergeAllLocsGCS", "DISABLED", "DISABLED", wgs84GCS) 

#Get an insert cursor on the CollarDeploymentLocations feature class
insertCursor = arcpy.InsertCursor(collarDeploymentLocsFC)

for i in eleNamesArray:
    print "Name: " + i
    #Get the list of unique name_chrono vals for a given name
    queryString="MovDataID='" + i + "'"
    chronos=list(set([newrow[0] for newrow in arcpy.da.SearchCursor(MovGDB_Path + "\\MergeAllLocsGCS",["CalcID"],
                                                                    where_clause=queryString)]))
    for j in chronos:
        #Now go through each chrono value
        queryString2="CalcID='" + j + "'"
        sqlPostfixString ="ORDER BY Fixtime ASC"
        fixes = [newrow for newrow in arcpy.da.SearchCursor(MovGDB_Path + "\\MergeAllLocsGCS",["CalcID","Fixtime","SHAPE@"],
                                                            where_clause=queryString2,sql_clause=("",sqlPostfixString))]
        if len(fixes) != 0:
            #print(len(fixes))
            firstFix=fixes[0]
            feat = insertCursor.newRow()
            feat.setValue("MovDataID", i)
            feat.setValue("CalcID", firstFix[0])
            feat.setValue("Fixtime", firstFix[1])
            feat.shape=firstFix[2]
            insertCursor.insertRow(feat)
        del(fixes)
    del(chronos)
del(insertCursor)

# Range Size Summary

In [ ]:
# Import the 90th percentile ETD Data
df16Day = pd.read_csv(filepath_or_buffer=etd16DayAnalysisTableFilt + '_new', header=0, parse_dates=True)
dfAnnual = pd.read_csv(filepath_or_buffer=etdAnnualAnalysisTableFilt + '_new', header=0, parse_dates=True)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of 16-Day ETD 90th percentile Ranges')
sns.distplot(df16Day.AreaKm, ax=ax)
plt.savefig(figuresFolder + r'/distributions/16Day_Range_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD 90th percentile Ranges')
sns.distplot(dfAnnual.AreaKm)
plt.savefig(figuresFolder + r'/distributions/Annual_Range_Dist.png')

#### Mean 16-day Range

In [ ]:
mean_16day_range = np.mean(df16Day.AreaKm)
mean_16day_range

In [ ]:
# equivalent square pixel cell dimension
np.sqrt(mean_16day_range * 1000000)

#### Minimum 16 Day Range

In [ ]:
df16Day.iloc[df16Day["AreaKm"].idxmin()]

#### Max 16-Day Range

In [ ]:
df16Day.iloc[df16Day["AreaKm"].idxmax()]

#### Mean Annual Range

In [ ]:
mean_annual_range = np.mean(dfAnnual.AreaKm)
mean_annual_range

In [ ]:
# equivalent square pixel cell dimension
np.sqrt(mean_annual_range * 1000000)

#### Min Annual Range

In [ ]:
dfAnnual.iloc[dfAnnual["AreaKm"].idxmin()]

#### Max Annual Range

In [ ]:
dfAnnual.iloc[dfAnnual["AreaKm"].idxmax()]

#### Range sumarized by metaregion

In [ ]:
def iqr(x):
    return np.subtract(*np.percentile(x, [75, 25]))

def percentile25(x):
    return np.percentile(x, 25)

def percentile75(x):
    return np.percentile(x, 75)

In [ ]:
df16Day.pivot_table(index=['MetaRegion'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

In [ ]:
dfAnnual.pivot_table(index=['MetaRegion'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

#### Range summarized by species

In [ ]:
df16Day.pivot_table(index=['Species'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

In [ ]:
dfAnnual.pivot_table(index=['Species'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

#### Range sumarized by sex

In [ ]:
df16Day.pivot_table(index=['Sex'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

In [ ]:
dfAnnual.pivot_table(index=['Sex'],
                            values=['AreaKm'],
                            aggfunc=[np.median, percentile25, percentile75, iqr], #np.mean, stats.sem,
                            fill_value='NA').T 

# ETD-MCP-KDE Range Size Comparison

In [ ]:
# Import the 90th percentile ETD Data
df16Day = pd.read_csv(filepath_or_buffer=etd16DayAnalysisTableFilt + '_new', header=0, parse_dates=True)
dfAnnual = pd.read_csv(filepath_or_buffer=etdAnnualAnalysisTableFilt + '_new', header=0, parse_dates=True)

In [ ]:
df16Day.columns

In [ ]:
# Import ETD ranges, MCP Ranges and KDE ranges
df16Day_ETD = df16Day[['CalcID','MetaRegion','country', 'region', 'Area', 'IntersectPercent' ]]
df16Day_ETD.set_index('CalcID', inplace=True, drop=True)
df16Day_ETD.columns=['Biome','Country', 'Site', 'ETD', 'PAI']

dfAnnual_ETD = dfAnnual[['CalcID','MetaRegion','country', 'region', 'Area', 'IntersectPercent']]
dfAnnual_ETD.set_index('CalcID', inplace=True, drop=True)
dfAnnual_ETD.columns=['Biome','Country', 'Site', 'ETD', 'PAI']

df16Day_KDE = pd.read_csv(filepath_or_buffer=RangeFolder + '\\range_comparison\\KDE16DayPolygonGCS.csv',
                          header=0)
df16Day_KDE = df16Day_KDE[['CalcID','Area']]
df16Day_KDE.set_index('CalcID', inplace=True, drop=True)
df16Day_KDE.columns=['KDE']

dfAnnual_KDE = pd.read_csv(filepath_or_buffer=RangeFolder + '\\range_comparison\\KDEAnnualPolygonGCS.csv',
                           header=0)
dfAnnual_KDE = dfAnnual_KDE[['CalcID','Area']]
dfAnnual_KDE.set_index('CalcID', inplace=True, drop=True)
dfAnnual_KDE.columns=['KDE']

df16Day_MCP = pd.read_csv(filepath_or_buffer=RangeFolder + '\\range_comparison\\MCP16DayPolygonGCS.csv',
                          header=0)
df16Day_MCP = df16Day_MCP[['CalcID','MCPArea']]
df16Day_MCP.set_index('CalcID', inplace=True, drop=True)
df16Day_MCP.columns=['MCP']

dfAnnual_MCP = pd.read_csv(filepath_or_buffer=RangeFolder + '\\range_comparison\\MCPAnnualPolygonGCS.csv',
                           header=0)
dfAnnual_MCP = dfAnnual_MCP[['CalcID','MCPArea']]
dfAnnual_MCP.set_index('CalcID', inplace=True, drop=True)
dfAnnual_MCP.columns=['MCP']

In [ ]:
all_Annual_ranges = dfAnnual_ETD.join(dfAnnual_KDE, how='inner').join(dfAnnual_MCP, how='inner')
all_Annual_ranges['timespan'] = 'Annual'
all_16Day_ranges = df16Day_ETD.join(df16Day_KDE, how='inner').join(df16Day_MCP, how='inner')
all_16Day_ranges['timespan'] = '16Day'

In [ ]:
allranges=pd.concat([all_16Day_ranges, all_Annual_ranges])
allranges['ETD'] = allranges['ETD']/1000000
allranges['KDE'] = allranges['KDE']/1000000
allranges['MCP'] = allranges['MCP']/1000000

In [ ]:
allranges.head()

In [ ]:
range_summary_pivot = allranges.pivot_table(index=['Biome','Country','Site','timespan'],
                            values=['KDE', 'MCP', 'ETD', 'PAI'],
                            aggfunc=[max, np.median, np.mean, min],
                            fill_value='NA').reset_index()

In [ ]:
range_summary_pivot.to_csv(tablesFolder + 'range_size_pivot_table.csv')

# Covariate Data Summary

In [ ]:
def cov_summary(vals):
    result = OrderedDict()
    result['0%'] = np.min(vals)
    result['5%'] = np.percentile(vals, 5.)
    result['25%'] =  np.percentile(vals, 25.)
    result['50%'] = np.percentile(vals, 50.)
    result['75%'] = np.percentile(vals, 75.)
    result['95%'] = np.percentile(vals, 95.)
    result['100%'] = np.max(vals)
    result['mean'] = np.mean(vals)
    result['sd'] = np.std(vals)
    return pd.Series(result)

In [ ]:
# 16-Day Range Covariate Summary
df16Day[['AreaKm','ndviVal','treeVal','trmmVal','tempVal','waterVal','slopeVal','hfVal','IntersectPercent']] \
 .apply(cov_summary).T

In [ ]:
sns.distplot(df16Day.ndviVal)

In [ ]:
sns.distplot(df16Day.treeVal)

In [ ]:
sns.distplot(df16Day.trmmVal)

In [ ]:
sns.distplot(df16Day.tempVal)

In [ ]:
sns.distplot(df16Day.waterVal)

In [ ]:
sns.distplot(df16Day.slopeVal)

In [ ]:
sns.distplot(df16Day.hfVal)

In [ ]:
sns.distplot(df16Day.IntersectPercent)

## Annual Range & Covariate Summary

In [ ]:
dfAnnual.pivot_table(index=['MetaRegion'],
                            values=['AreaKm'],
                            aggfunc=[np.percentile(25),np.percentile(50),np.percentile(75)],
                            fill_value='NA').T #.to_csv(tablesFolder + '\\Covariate_stats_by_metaregion_annual.csv')

In [ ]:
dfAnnual[['AreaKm','ndviVal','treeVal','trmmVal','tempVal','waterVal','slopeVal','hfVal','IntersectPercent']]. \
apply(func=cov_summary, axis=0).T

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD NDVI Values')
sns.distplot(dfAnnual.ndviVal)
plt.savefig(figuresFolder + r'/distributions/Annual_NDVI_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Tree Cover Values')
sns.distplot(dfAnnual.treeVal)
plt.savefig(figuresFolder + r'/distributions/Annual_Tree_Cover_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Rainfall Rate Values')
sns.distplot(dfAnnual.trmmVal)
plt.savefig(figuresFolder + r'/distributions/Annual_Rainfall_Rate_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Land Surface Temperature Values')
sns.distplot(dfAnnual.tempVal)
plt.savefig(figuresFolder + r'/distributions/Annual_LST_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Water Occurence Values')
sns.distplot(dfAnnual.waterVal)
plt.savefig(figuresFolder + r'/distributions/Annual_WATER_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Slope Values')
sns.distplot(dfAnnual.slopeVal)
plt.savefig(figuresFolder + r'/distributions/Annual_SLOPE_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Human Footprint Values')
sns.distplot(dfAnnual.hfVal)
plt.savefig(figuresFolder + r'/distributions/Annual_HF_Dist.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,8)
ax.set_title('Distribution of Annual ETD Protected Area Intersection Values')
sns.distplot(dfAnnual.IntersectPercent)
plt.savefig(figuresFolder + r'/distributions/Annual_PAI_Dist.png')

#### Max HF Value - Annual

In [ ]:
dfAnnual.iloc[dfAnnual["hfVal"].idxmax()]

#### Max HF Value 16-Day 

In [ ]:
df16Day.iloc[df16Day["hfVal"].idxmax()]

#### Min HF Value - Annual

In [ ]:
dfAnnual.iloc[dfAnnual["hfVal"].idxmin()]

#### Max Water Val

In [ ]:
df16Day.iloc[df16Day["waterVal"].idxmax()]

In [ ]:
dfAnnual.iloc[dfAnnual["waterVal"].idxmax()]

# Does Temperature variance change with temperature

In [ ]:
# Does tempeprature variance increase with temperature?
temp_variance = df16Day['tempVal'].groupby(pd.cut(df16Day.tempVal, np.arange(10, 52, 2))).agg(
    {'mean': np.mean, 'var': np.var,})
plt.figure()
temp_variance.plot()

### Join MCP, Path Metrics to Annual DF

In [ ]:
# Annual ETD ranges Dataframe
dfAnnual = pd.read_csv(filepath_or_buffer=etdAnnualAnalysisTableFilt,header=0, parse_dates=True)
dfAnnual['ETD90Area'] = dfAnnual.AreaKm
dfAnnual.index = dfAnnual.CalcID
len(dfAnnual.index)

In [ ]:
dfAnnualMCP100 = ConvertFeatureClassToPandasDataFrame(mcpRangesAnnual_Path + "\\" + 'MCPAnnualPolygonsGCS')
dfAnnualMCP100 = dfAnnualMCP100.set_index('CalcID')
dfAnnualMCP100=dfAnnualMCP100[dfAnnualMCP100["ChosenPercentile"]==100][['MCPArea']]
dfAnnualMCP100['MCP100Area'] = dfAnnualMCP100.MCPArea/1000000
dfAnnualMCP100 = dfAnnualMCP100[['MCP100Area']]
#dfAnnualMCP100.head()

In [ ]:
dfAnnualETD100 = ConvertFeatureClassToPandasDataFrame(etdRangesAnnual_Path + "\\" + 'ETDAnnualPolygonsGCS')
dfAnnualETD100 = dfAnnualETD100.set_index('CalcID')
dfAnnualETD100=dfAnnualETD100[dfAnnualETD100["DesiredPercentile"]==1][['Area']]
dfAnnualETD100['ETD100Area'] = dfAnnualETD100.Area/1000000
dfAnnualETD100 = dfAnnualETD100[['ETD100Area']]
#dfAnnualETD100.head()

In [ ]:
dfAnnualPaths = ConvertFeatureClassToPandasDataFrame(trajPathMetricsAnnual_Path + "\\TrajPathMetrics")
dfAnnualPaths = dfAnnualPaths.set_index('CalcID')
dfAnnualPaths = dfAnnualPaths[['DisplMtrs','MaxDisplMtrs','PathDistMtrs','Tortuosity','SpeedKmHrs']]
dfAnnualPaths['PathDistKm'] = dfAnnualPaths['PathDistMtrs']/1000

In [ ]:
dfAnnual = dfAnnual.join(dfAnnualETD100).join(dfAnnualMCP100).join(dfAnnualPaths)
len(dfAnnual.index)

In [ ]:
dfAnnual.to_csv(path_or_buf=AnnualAnalysisTable, date_format='%Y-%m-%d %H:%M:%S',index=False)

In [ ]:
del(dfAnnualETD100)
del(dfAnnualMCP100)
del(dfAnnualPaths)

#### Summarize range statistics by Meta-Region

In [ ]:
rangeSizeSummary  =  dfAnnual.pivot_table(index=['MetaRegion'], values=['ETD90Area','ETD100Area','MCP100Area'],aggfunc=[min,np.mean,max])
rangeSizeSummary.to_csv(tablesFolder + '\\RangeSizeSummary.csv')

In [ ]:
print AnnualAnalysisTable

# Compare MCP-ETD-KDE Ranges

In [ ]:
dfAnnual = pd.read_csv(AnnualAnalysisTable, parse_dates=True, header=0)
len(dfAnnual.index)

In [ ]:
# log transform the ETD and MCP ranges
dfAnnual['log_MCP100Area'] = np.log(dfAnnual.MCP100Area)
dfAnnual['log_ETD100Area'] = np.log(dfAnnual.ETD100Area)

In [ ]:
# calculate the ETD/MCP ratio
dfAnnual['MCP/ETD Ratio'] = dfAnnual['MCP100Area']/dfAnnual['ETD100Area']

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(1, figsize=(15, 7))
    g = sns.scatterplot(x='log_MCP100Area', y='log_ETD100Area', data=dfAnnual, hue='MetaRegion', size='MCP/ETD Ratio',
                        sizes=(40, 600), ax=ax, alpha=0.8, legend='brief')\n
    ax.set_ylabel(\"Log - ETD Area (Sq.Km)\")
    ax.set_xlabel(\"Log - MCP Area (Sq.Km)\")
    ax.set_xlim(2,13)
    ax.set_ylim(2,10)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)
    print(type(handles[0]))

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(1, figsize=(15, 7))
    g = sns.scatterplot(x='MCP100Area', y='ETD100Area', hue="MetaRegion", data=dfAnnual, s=80, ax=ax, alpha=0.8)
    ax.set_ylabel("ETD Area (Sq.Km)")
    ax.set_xlabel("MCP Area (Sq.Km)")
    ax.set_xlim(0,35000)
    ax.set_ylim(0,3000)

In [ ]:
with plt.style.context('bmh'):
    fig = plt.figure(figsize=(15,7))
    title = fig.suptitle(\"ETD/MCP Area Ratio by MetaRegion\", fontsize=14)
    ax = fig.add_subplot(1,1,1)
    g = sns.FacetGrid(data=dfAnnual, hue='MetaRegion')
    #g.map(sns.kdeplot, 'ETD_MCP_Ratio', kernel='tri', ax=ax)
    g.map(sns.distplot,'MCP/ETD Ratio', kde=False, norm_hist=False, bins=30, ax=ax)
    ax.legend(title='MetaRegion')
    plt.close(2)

# Modeling ETD Range Size against covariates (R)

## 16-Day Range Modeling

In [ ]:
%%R

### Initial R package import
# install.packages(c('lattice','nlme','mgcv', 'MASS', 'MuMIn', 'ggplot2', 'gridExtra'))

library(lattice)
library(nlme)
library(mgcv)
library(MASS)
library(MuMIn)
library(ggplot2)
library(gridExtra)
theme_set(theme_gray(base_size=10))

ctrl <- lmeControl(opt='optim')

# Set working directory
setwd('C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/Analysis Worksheet')

#Read in the 16-Day PanAfEl16Day Data
PanAfEl16Day<-read.csv("../../Tables/ETD_16Day_Analysis_Table_Filtered.csv_new",header=T)
nrow(PanAfEl16Day) #10322

PanAfElAnnual<-read.csv("../../Tables/ETD_Annual_Analysis_Table_Filtered.csv",header=T)
nrow(PanAfElAnnual) #302

###Calculate Days From Start Function Definition
calcTemporalLag<-function(movDataID,inputDF)
{
    #Calculate the earliest date
    minDate<-min(inputDF[inputDF$MovDataID == movDataID,"StartDate2"])
    
    #Calculate the difference in days for 
    inputDF[inputDF$MovDataID == movDataID,"Days"]<-difftime(inputDF[inputDF$MovDataID == movDataID,"StartDate2"],minDate,units="days")
    
    #Calculate difference in years
    inputDF[inputDF$MovDataID == movDataID,"Years"]<-inputDF[inputDF$MovDataID == movDataID,"Days"]/365.25
    
    return(inputDF)
}

#panel.cor for use with the pairs function
panel.cor <- function(x, y, digits = 2, prefix = "", cex.cor, ...) 
{ 
  usr <- par("usr"); on.exit(par(usr)) 
  par(usr = c(0, 1, 0, 1)) 
  r <- abs(cor(x, y)) 
  txt <- format(c(r, 0.123456789), digits = digits)[1] 
  txt <- paste0(prefix, txt) 
  if(missing(cex.cor)) cex.cor <- 0.8/strwidth(txt) 
  text(0.5, 0.5, txt, cex = cex.cor * r) 
} 

panel.hist <- function(x, ...)
{
    usr <- par("usr"); on.exit(par(usr))
    par(usr = c(usr[1:2], 0, 1.5) )
    h <- hist(x, plot = FALSE)
    breaks <- h$breaks; nB <- length(breaks)
    y <- h$counts; y <- y/max(y)
    rect(breaks[-nB], 0, breaks[-1], y, col="cyan", ...)
}

DiagnosticPlot<-function(theModel,yObserved,timeVals,modelLevel,outputFigureName)
{
    #theModel.resid.1<-resid(theModel,level=modelLevel, type="normalized")
    theModel.resid.1<-resid(theModel,level=modelLevel)
    theModel.yhat.1<-fitted(theModel,level=modelLevel) #note there is a subtle diff between fitted and predicted commands
    #Get the Model name
    model.name<-deparse(substitute(theModel))

    #Setup the png output file
    #pdf(file=outputFigureName,width=8,height=5,pointsize=12,family="Times") #paste(model.name,"_DiagnosticPlot.pdf")
    png(file=outputFigureName,width=8,height=5,family="Times", res=300, units = 'in')
    par(mfrow=c(2,2),mai=c(0.6,0.6,0.6,0.6),cex=0.55)

    #Note the order of these plots mimics the gam.fit function output
    #Graph 1: QQ Plot of model residuals
    qqnorm(theModel.resid.1,main=paste("Model",model.name,", Normality plot",sep=" "))

    #Graph 2: Linear Predictor (X) Vs Model Residuals (Y)
    plot(theModel.yhat.1,theModel.resid.1, main=paste("Model",model.name,", Residual Plot",sep=" "),
      xlab="Predicted Value (log(Area Sq.Km))", ylab="residual")

    #Graph 3: Histogram of model residuals
    hist(theModel.resid.1, breaks=8 , density=10,col="green", border="black",xlab=paste("Model",model.name,"residuals (level 1)",sep=" "),
         main=paste("Model",model.name,", Error Distribution",sep=" "))

    #Graph 4: Observed values Vs Model Predicted values
    plot(theModel.yhat.1,yObserved, main=paste("Model",model.name,", yhat vs log(Area Sq.Km)",sep=" "),
         xlab="Predicted Value", ylab=paste("Observed Value",model.name,sep=" "))
    abline(0,1,col='red')

    #Graph: Observed times values Vs Model residuals
    #plot(timeVals,theModel.resid.1, main=paste("Model",model.name,", Residual vs Days",sep=" "),xlab="Days", ylab="residual",pch=8)

    par(mfrow=c(1,1),mai=c(1.0,1.0,1.0,1.0),cex=1.0)

    dev.off()
}

#Function to retrieve legend from ggplot object
get_legend<-function(myggplot){
  tmp <- ggplot_gtable(ggplot_build(myggplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}
                      
mround <- function(x,base){ 
        base*round(x/base) 
}

get_CI <- function(y,pref="") {
    r1 <- t(apply(y,1,quantile,c(0.025,0.975)))
    setNames(as.data.frame(r1),paste0(pref,c("lwr","upr")))
}

standardize<-function(x){
    s.x <- scale(x)
    return(s.x)
}

unstandardize<-function(s.x, x){
    scl <- scale(x)
    return(s.x * attr(scl, 'scaled:scale') + attr(scl, 'scaled:center'))
}

model_covars<-function(model){
    # return the names of the model continuous covariates
    covars <-as.character(attr(terms(model), "term.labels"))
    covars <- na.omit(gsub("^\\w+:(\\w.*)", NA, covars)) # regex match to any character sequence followed by a colon followed by character sequence
    covars <- covars[! covars %in% names(lapply(model$contrasts, rownames))]
    return(covars)
}

model_predict <- function(model, dataF, yaxis_spacing=25)
{
    minmax_y<-c(Inf,0) # create a dummy global y-axis range that will be adjusted depending on the data
    xn = 100 # the number of x samples to make predictions at
    bn = 10000 # the number of times to resample the model coefficients

    # Pick n random draws from a nultivariate normal distribution defined by a mean vector (estimated model parameter values) 
    #and variance-covariance matrix (estimated parameter variances are on the diagonal). 
    # So we are effectively choosing bn realizations of possible model parameters based on their estimated mean and variance values
    params_resamp <- mvrnorm(n=bn, mu = fixef(model), Sigma = vcov(model))

    # Get the names of the model continuous covariates
    covars <- model_covars(model)
    print(covars)

    plots <-vector("list", length(covars)+1) # create an empty vector to hold plot objects as they are created
    j=1

    for(covar in covars){
        print(covar)
        # start a new panel in the prediction figure
        covars_ <- covars[! covars %in% covar]  # vector of covariate names other than the current covariate

        # Create a list of xn values ranging from the min to the max for the given covariate. 
        x_range <- setNames(list(seq(min(standardize(dataF[covar])), max(standardize(dataF[covar])), length.out=xn)), covar)

        # Create a dataframe where each combination of factor is represented with the covariate ranging from it's min to max value 
        newdata <- expand.grid(c(lapply(model$contrasts, rownames), x_range))

        # add the other covariates to the dataframe held at the 0 level (their mean if they are centered)
        newdata[covars_]<-0 

        # use the predictor variables (RHS) from the model formula and create the model matrix including all interactions, contrasts and dummy coding
        model_matrix <- model.matrix(eval(model$call$fixed)[-2], data=newdata)

        pred_func <- function(B){return(model_matrix %*% B)}

        # predict mean response based on the REML value of regression coefficients
        newdata["predict_y"] = predict(model, newdata, level=0)

        # use resampled regression coefficients to generate confidence intervals for the regression line
        yvals <- apply(params_resamp, 1, pred_func)

        c1 <- get_CI(yvals)

        newdata["predict_y_upr"] = c1["upr"]
        newdata["predict_y_lwr"] = c1["lwr"]
        newdata["y"] = as.numeric(unlist(exp(newdata["predict_y"])))
        newdata["y_upr"] = as.numeric(unlist(exp(newdata["predict_y_upr"])))
        newdata["y_lwr"] = as.numeric(unlist(exp(newdata["predict_y_lwr"])))

        # For plotting, need to transform the standardized x range back to the original scale
        newdata$xvals <- as.numeric(unlist(unstandardize(newdata[covar], dataF[gsub('s.','',covar)]))) # assumes the dataframe uses the s. prefix for standardized variables

        # adjust the y-axis
        minmax_y[1] = min(newdata["y"],newdata["y_lwr"])
        minmax_y[2] = max(newdata["y"],newdata["y_upr"])

        print(minmax_y)
        
        newdata$interact <- interaction(newdata[names(model$contrasts)])

        #newdata <- na.omit(newdata)
        
        p<-ggplot(data=newdata, aes(x=xvals, y=y, group=interact, colour=interact)) + geom_line(size=0.5) + 
        labs(x=gsub('s.','',covar), y="Range Area (Sq.Km)", title="") + theme(legend.title=element_blank()) +
        geom_line(data=newdata, aes(x=xvals, y=y_upr, group=interact, colour=interact), size=0.5, linetype="dotted") +
        geom_line(data=newdata, aes(x=xvals, y=y_lwr, group=interact, colour=interact), size=0.5, linetype="dotted")

        #Use a common scale for the y-axis
        plots[[length(covars)+1]] = get_legend(p)
        plots[[j]] = p
        j=j+1
    }

    #Use a common scale for the y-axis
    minmax_y<-c(mround(minmax_y[1],5), mround(minmax_y[2],5))
    breaks <- seq(minmax_y[1], minmax_y[2], by=yaxis_spacing)
    for(i in 1:length(covars))
    {
        plots[[i]] = plots[[i]] + scale_y_continuous(limits=minmax_y, breaks=breaks, labels=breaks)
        plots[[i]] = plots[[i]] + theme(legend.position = "none")
    }
    
    return(plots)
}

# 16-Day data setup & filters

#Convert the start/end time columns to datetime objects and append to the dataframe
PanAfEl16Day$StartDate2<-as.POSIXct(PanAfEl16Day$StartDate, format="%Y-%m-%d %H:%M:%S")
PanAfEl16Day$EndDate2<-as.POSIXct(PanAfEl16Day$EndDate, format="%Y-%m-%d %H:%M:%S")

#Calculate the temporal lag of each observation in both days and years
for(s in unlist(list(unique(PanAfEl16Day$MovDataID))))
{
    PanAfEl16Day<-calcTemporalLag(s,PanAfEl16Day)
}

#Order the data alphabetically and temporally
PanAfEl16Day<-PanAfEl16Day[order(PanAfEl16Day$MovDataID,PanAfEl16Day$Days),]

# rename elephant to 'savannah elephant'
PanAfEl16Day[which(PanAfEl16Day$Species=="Elephant"),]$Species <- "Savannah Elephant"

#Calculate factors
PanAfEl16Day$fSpecies<-factor(PanAfEl16Day$Species)
PanAfEl16Day$fSex<-factor(PanAfEl16Day$Sex)
PanAfEl16Day$fRegion<-factor(PanAfEl16Day$region)

#Log transformations following Zuur et al.(2009) pgs. 129 & 131
PanAfEl16Day$logArea<-log(PanAfEl16Day$AreaKm)


### Standardize covariates and names
#See http://warnercnr.colostate.edu/~gwhite/mark/markhelp/standardize_individual_covariates.htm

#NDVI
names(PanAfEl16Day)[names(PanAfEl16Day) == 'ndviVal'] <- 'NDVI'
PanAfEl16Day$s.NDVI<-standardize(PanAfEl16Day$NDVI)

#NDWI
names(PanAfEl16Day)[names(PanAfEl16Day) == 'ndwiVal'] <- 'NDWI'
PanAfEl16Day$s.NDWI<-standardize(PanAfEl16Day$NDWI)

#EVI
names(PanAfEl16Day)[names(PanAfEl16Day) == 'eviVal'] <- 'EVI'
PanAfEl16Day$s.EVI<-standardize(PanAfEl16Day$EVI)

#Temperature
names(PanAfEl16Day)[names(PanAfEl16Day) == 'tempVal'] <- 'LST'
PanAfEl16Day$s.LST<-standardize(PanAfEl16Day$LST)

#TreeCover
names(PanAfEl16Day)[names(PanAfEl16Day) == 'treeVal'] <- 'TREE'
PanAfEl16Day$s.TREE<-standardize(PanAfEl16Day$TREE)

#Water
names(PanAfEl16Day)[names(PanAfEl16Day) == 'waterVal'] <- 'WATER'
PanAfEl16Day$s.WATER<-standardize(PanAfEl16Day$WATER)

#TRMM
names(PanAfEl16Day)[names(PanAfEl16Day) == 'trmmVal'] <- 'TRMM'
PanAfEl16Day$s.TRMM<-standardize(PanAfEl16Day$TRMM)

#HF
names(PanAfEl16Day)[names(PanAfEl16Day) == 'hfVal'] <- 'HFI'
PanAfEl16Day$s.HFI<-standardize(PanAfEl16Day$HFI)

#Slope
names(PanAfEl16Day)[names(PanAfEl16Day) == 'slopeVal'] <- 'SLOPE'
PanAfEl16Day$s.SLOPE<-standardize(PanAfEl16Day$SLOPE)

#PAIntersect
names(PanAfEl16Day)[names(PanAfEl16Day) == 'IntersectPercent'] <- 'PAI'
PanAfEl16Day$s.PAI<-standardize(PanAfEl16Day$PAI)

#TGI
PanAfEl16Day$s.TGI<-standardize(PanAfEl16Day$TGI)


## Annual Data setup

#Convert the start/end time columns to datetime objects and append to the dataframe
PanAfElAnnual$StartDate2<-as.POSIXct(PanAfElAnnual$StartDate, format="%Y-%m-%d %H:%M")
PanAfElAnnual$EndDate2<-as.POSIXct(PanAfElAnnual$EndDate, format="%Y-%m-%d %H:%M")

#Calculate the temporal lag of each observation in both days and years
for(s in unlist(list(unique(PanAfElAnnual$MovDataID))))
{
    PanAfElAnnual<-calcTemporalLag(s,PanAfElAnnual)
}

#Order the data alphabetically and temporally
PanAfElAnnual<-PanAfElAnnual[order(PanAfElAnnual$MovDataID,PanAfElAnnual$Days),]

# rename elephant to 'savannah elephant'
PanAfElAnnual[which(PanAfElAnnual$Species=="Elephant"),]$Species <- "Savannah Elephant"

#Calculate factors
PanAfElAnnual$fSpecies<-factor(PanAfElAnnual$Species)
PanAfElAnnual$fSex<-factor(PanAfElAnnual$Sex)
PanAfElAnnual$fRegion<-factor(PanAfElAnnual$region)

#Create an interaction term between sex and MetaRegion
PanAfElAnnual$SexRegion<-interaction(PanAfElAnnual$Sex,PanAfElAnnual$MetaRegion)

#transformation following Zuur et al.(2009) pgs. 129 & 131
PanAfElAnnual$logArea<-log(PanAfElAnnual$AreaKm)

### Standardize covariates and names

#NDVI
names(PanAfElAnnual)[names(PanAfElAnnual) == 'ndviVal'] <- 'NDVI'
PanAfElAnnual$s.NDVI<-standardize(PanAfElAnnual$NDVI)

#NDWI
names(PanAfElAnnual)[names(PanAfElAnnual) == 'ndwiVal'] <- 'NDWI'
PanAfElAnnual$s.NDWI<-standardize(PanAfElAnnual$NDWI)

#EVI
names(PanAfElAnnual)[names(PanAfElAnnual) == 'eviVal'] <- 'EVI'
PanAfElAnnual$s.EVI<-standardize(PanAfElAnnual$EVI)

#Temperature
names(PanAfElAnnual)[names(PanAfElAnnual) == 'tempVal'] <- 'LST'
PanAfElAnnual$s.LST<-standardize(PanAfElAnnual$LST)

#TreeCover
names(PanAfElAnnual)[names(PanAfElAnnual) == 'treeVal'] <- 'TREE'
PanAfElAnnual$s.TREE<-standardize(PanAfElAnnual$TREE)

#Water
names(PanAfElAnnual)[names(PanAfElAnnual) == 'waterVal'] <- 'WATER'
PanAfElAnnual$s.WATER<-standardize(PanAfElAnnual$WATER)

#TRMM
names(PanAfElAnnual)[names(PanAfElAnnual) == 'trmmVal'] <- 'TRMM'
PanAfElAnnual$s.TRMM<-standardize(PanAfElAnnual$TRMM)

#HF
names(PanAfElAnnual)[names(PanAfElAnnual) == 'hfVal'] <- 'HFI'
PanAfElAnnual$s.HFI<-standardize(PanAfElAnnual$HFI)

#Slope
names(PanAfElAnnual)[names(PanAfElAnnual) == 'slopeVal'] <- 'SLOPE'
PanAfElAnnual$s.SLOPE<-standardize(PanAfElAnnual$SLOPE)

#PAIntersect
names(PanAfElAnnual)[names(PanAfElAnnual) == 'IntersectPercent'] <- 'PAI'
PanAfElAnnual$s.PAI<-standardize(PanAfElAnnual$PAI)

#TGI
PanAfElAnnual$s.TGI<-standardize(PanAfElAnnual$TGI)


## Covariate Plots

#FIGURE: 16Day_Range_Covariate_PairPlot - Pair Plot of explanatory variables
PanAfEl16DayXCov<-cbind(PanAfEl16Day$fSex,
                        PanAfEl16Day$fSpecies,
                        PanAfEl16Day$NDVI,
                        PanAfEl16Day$NDWI,
                        PanAfEl16Day$EVI,
                        PanAfEl16Day$LST,
                        PanAfEl16Day$TRMM,
                        PanAfEl16Day$TREE,
                        PanAfEl16Day$WATER,
                        PanAfEl16Day$HFI,
                        PanAfEl16Day$PAI,
                        PanAfEl16Day$SLOPE)
colnames(PanAfEl16DayXCov)<-c('SEX','SPECIES','NDVI','NDWI','EVI','LST','TRMM','TREE','WATER','SLOPE','HFI','PAI')

# pdf(file="../../Figures/16Day_Range_Covariate_PairPlot.pdf",width=6,height=6,pointsize=12,family="Times")
# pairs(PanAfEl16DayXCov,lower.panel=panel.smooth,upper.panel=panel.cor,diag.panel=panel.hist)
# dev.off()

png(file="../../Figures/16Day_Range_Covariate_PairPlot.png",width=6,height=6,res=300, units='in')
pairs(PanAfEl16DayXCov,lower.panel=panel.smooth,upper.panel=panel.cor,diag.panel=panel.hist)
dev.off()

#FIGURE: Annual_Range_Covariate_PairPlot.pdf - Pair Plot of explanatory variables
PanAfElAnnualXCov<-cbind(PanAfElAnnual$fSex,
                         PanAfElAnnual$fSpecies,
                         PanAfElAnnual$NDVI,
                         PanAfElAnnual$NDWI,
                         PanAfElAnnual$EVI,
                         PanAfElAnnual$LST,
                         PanAfElAnnual$TRMM,
                         PanAfElAnnual$TREE,
                         PanAfElAnnual$WATER,
                         PanAfElAnnual$HFI,
                         PanAfElAnnual$PAI,
                         PanAfElAnnual$SLOPE)
colnames(PanAfElAnnualXCov)<-c('SEX','SPECIES','NDVI','NDWI','EVI','LST','TRMM','TREE','WATER','HFI','PAI','SLOPE')

# pdf(file="../../Figures/Annual_Range_Covariate_PairPlot.pdf",width=6,height=6,pointsize=12,family="Times")
# pairs(PanAfElAnnualXCov,lower.panel=panel.smooth,upper.panel=panel.cor,diag.panel=panel.hist)
# dev.off()

png(file="../../Figures/Annual_Range_Covariate_PairPlot.png",width=6,height=6,res=300, units='in')
pairs(PanAfElAnnualXCov,lower.panel=panel.smooth,upper.panel=panel.cor,diag.panel=panel.hist)
dev.off()

In [ ]:
%%R

## Modeling procedure based on 'Top-Down' described by West et al 2014 pg. 39.

### STEP 1: Start with a well-specified mean structure

# Colinearity between s.NDVI, s.NDWI, s.EVI so need to choose best model

# s.NDVI model
f_full_s.NDVI <- formula(logArea~1 + Sex + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full_s.NDVI<-lme(f_full_s.NDVI, random=~1|fRegion/MovDataID, data=PanAfEl16Day, method='ML',
                   correlation=corCAR1(form=~Years|fRegion/MovDataID))

# s.EVI model
f_full_s.EVI <- formula(logArea~1 + Sex + Species + s.EVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full_s.EVI<-lme(f_full_s.EVI, random=~1|fRegion/MovDataID, data=PanAfEl16Day, method='ML',
                  correlation=corCAR1(form=~Years|fRegion/MovDataID))

# s.NDWI model
f_full_s.NDWI <- formula(logArea~1 + Sex + Species + s.NDWI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full_s.NDWI<-lme(f_full_s.NDWI, random=~1|fRegion/MovDataID, data=PanAfEl16Day, method='ML',
                   correlation=corCAR1(form=~Years|fRegion/MovDataID))
  
results<-model.sel(m_full_s.NDVI, m_full_s.EVI, m_full_s.NDWI)
results # Model selection favours m_full_s.NDVI 

"""
Model selection table 
              (Int)     s.HFI   s.NDV    s.PAI     s.SLO    s.LST      s.TRE
m_full_s.NDVI 2.874 -0.03466 0.03173 -0.03997 -0.009853 0.017750 -0.0001751
m_full_s.EVI  2.874 -0.03480         -0.03974 -0.009566 0.011340  0.0033610
m_full_s.NDWI 2.871 -0.03543         -0.03965 -0.009214 0.009605  0.0048670
                 s.TRM  s.WAT Sex Spc   s.EVI   s.NDW             family df
m_full_s.NDVI 0.007597 0.1643   +   +                 gaussian(identity) 15
m_full_s.EVI  0.007496 0.1644   +   + 0.01864         gaussian(identity) 15
m_full_s.NDWI 0.007124 0.1644   +   +         0.01655 gaussian(identity) 15
                 logLik    AICc delta weight
m_full_s.NDVI -9371.128 18772.3  0.00  0.780
m_full_s.EVI  -9372.983 18776.0  3.71  0.122
m_full_s.NDWI -9373.203 18776.5  4.15  0.098
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
"""

### STEP 2: Select a structure for the random effects in the model
## Select a structure for the random effects. Use REML here as we are testing random effects structures
m_gls_0 <- gls(f_full_s.NDVI, data=PanAfEl16Day, method='REML', control=ctrl) # See pg. 93 of West et al. 
m_rand_1 <- lme(f_full_s.NDVI, random=~1|fRegion, data=PanAfEl16Day, method='REML', control=ctrl)
m_rand_2 <- lme(f_full_s.NDVI, random=~1|MovDataID, data=PanAfEl16Day, method='REML', control=ctrl)
m_rand_3 <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1), data=PanAfEl16Day, method='REML', control=ctrl)
results <-model.sel(m_gls_0, m_rand_1, m_rand_2, m_rand_3)
results # Model selection favours m_rand_3
'''
Model selection table 
         (Int)     s.HFI   s.NDV    s.PAI     s.SLO     s.LST    s.TRE     s.TRM  s.WAT Sex Spc             family class  random df    logLik    AICc   delta weight
m_rand_3 2.863 -0.04213 0.04205 -0.05319  0.011720 -0.016290 -0.02982 -0.009721 0.2062   +   + gaussian(identity)   lme f+M%i%f 14 -11051.22 22130.5    0.00      1
m_rand_2 2.994 -0.04104 0.04146 -0.04725  0.005336 -0.014270 -0.02423 -0.009575 0.2032   +   + gaussian(identity)   lme       M 13 -11097.23 22220.5   90.00      0
m_rand_1 2.852 -0.09973 0.05459 -0.04511 -0.027760  0.003221 -0.01828 -0.010930 0.1738   +   + gaussian(identity)   lme       f 13 -12026.21 24078.5 1947.96      0
m_gls_0  3.023 -0.10940 0.04855  0.03920 -0.137800  0.055720  0.03570 -0.024610 0.1536   +   + gaussian(identity)   gls         12 -12610.32 25244.7 3114.18      0
Models ranked by AICc(x) 
Random terms: 
f = ‘1 | fRegion’
M%i%f = ‘1 | MovDataID %in% fRegion’
M = ‘1 | MovDataID’
'''

## Select a correlation structure for the residuals
m_rand_3_cor <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                    correlation=corCAR1(form=~Years), data=PanAfEl16Day, method='REML', control=ctrl)
results <-model.sel(m_rand_3, m_rand_3_cor)
results # m_rand_3_cor is selected

'''
Model selection table 
             (Int)     s.HFI   s.NDV    s.PAI     s.SLO    s.LST      s.TRE     s.TRM  s.WAT Sex Spc             family     correlation df     logLik    AICc   delta
m_rand_3_cor 2.873 -0.03458 0.03171 -0.04006 -0.009554  0.01762 -0.0006426  0.007578 0.1645   +   + gaussian(identity) corCAR1(~Years) 15  -9403.885 18837.8    0.00
m_rand_3     2.863 -0.04213 0.04205 -0.05319  0.011720 -0.01629 -0.0298200 -0.009721 0.2062   +   + gaussian(identity)                 14 -11051.223 22130.5 3292.67
             weight
m_rand_3_cor      1
m_rand_3          0
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
'''

## select a covariance structure for the residuals
m_rand_3_cor_weights_none <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                 correlation=corCAR1(form=~Years),
                                 data=PanAfEl16Day, method='REML', control=ctrl)
m_rand_3_cor_weights_region <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                   weights = varIdent(form = ~1 | fRegion),
                                   correlation=corCAR1(form=~Years),
                                   data=PanAfEl16Day, method='REML', control=ctrl)
# m_rand_3_cor_weights_sex is throwing an error - not sure why? 
# m_rand_3_cor_weights_sex <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
#                                 weights = varIdent(form = ~1 | fSex),
#                                 correlation=corCAR1(form=~Years),
#                                 data=PanAfEl16Day, method='REML', control=ctrl)
m_rand_3_cor_weights_species <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                    weights = varIdent(form = ~1 | fSpecies),
                                    correlation=corCAR1(form=~Years),
                                    data=PanAfEl16Day, method='REML', control=ctrl)
results <-model.sel(m_rand_3_cor_weights_none, m_rand_3_cor_weights_region,
                     m_rand_3_cor_weights_species)
results # Model m_rand_3_cor_weights_region is selected

"""
Model selection table 
                             (Int)     s.HFI   s.NDV    s.PAI     s.SLO   s.LST      s.TRE    s.TRM  s.WAT Sex Spc             family               weights df
m_rand_3_cor_weights_region  2.869 -0.03217 0.03553 -0.03737 -0.011340 0.02116 -0.0015590 0.008956 0.1645   +   + gaussian(identity)  varIdent(~1|fRegion) 33
m_rand_3_cor_weights_none    2.873 -0.03458 0.03171 -0.04006 -0.009554 0.01762 -0.0006426 0.007578 0.1645   +   + gaussian(identity)                       15
m_rand_3_cor_weights_species 2.872 -0.03435 0.03191 -0.04041 -0.009375 0.01772 -0.0011060 0.007657 0.1653   +   + gaussian(identity) varIdent(~1|fSpecies) 16
                                logLik    AICc  delta weight
m_rand_3_cor_weights_region  -9332.027 18730.3   0.00      1
m_rand_3_cor_weights_none    -9403.885 18837.8 107.54      0
m_rand_3_cor_weights_species -9403.332 18838.7 108.44      0
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’"""


In [ ]:
%%R

## Step 4: competing model selection

f_sex <- formula(logArea~1 + Sex)
m_sex <- lme(f_sex, random=list(fRegion=~1, MovDataID=~1),
              weights = varIdent(form = ~1 | fRegion),
              correlation=corCAR1(form=~Years),
              data=PanAfEl16Day, method='ML', control=ctrl)

f_species <- formula(logArea~1 + Species)
m_species <- lme(f_species, random=list(fRegion=~1, MovDataID=~1),
              weights = varIdent(form = ~1 | fRegion),
              correlation=corCAR1(form=~Years),
              data=PanAfEl16Day, method='ML', control=ctrl)


f_indv <- formula(logArea~1 + Sex + Species)
m_indv <- lme(f_indv, random=list(fRegion=~1, MovDataID=~1),
              weights = varIdent(form = ~1 | fRegion),
              correlation=corCAR1(form=~Years),
              data=PanAfEl16Day, method='ML', control=ctrl)

f_anthro <- formula(logArea~1 + s.HFI + s.PAI)
m_anthro <-lme(f_anthro, random=list(fRegion=~1, MovDataID=~1),
               weights = varIdent(form = ~1 | fRegion),
               correlation=corCAR1(form=~Years),
               data=PanAfEl16Day, method='ML', control=ctrl)

f_anthro_indv <-formula(logArea~1 + Sex + Species + s.HFI + s.PAI)
m_anthro_indv<-lme(f_anthro_indv, random=list(fRegion=~1, MovDataID=~1),
                   weights = varIdent(form = ~1 | fRegion),
                   correlation=corCAR1(form=~Years),
                   data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic <- formula(logArea~1 + s.NDVI + s.TREE)
m_biotic<-lme(f_biotic, random=list(fRegion=~1, MovDataID=~1),
              weights = varIdent(form = ~1 | fRegion),
              correlation=corCAR1(form=~Years),
              data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_indv <- formula(logArea~1 + Sex + Species + s.NDVI + s.TREE)
m_biotic_indv<-lme(f_biotic_indv, random=list(fRegion=~1, MovDataID=~1),
                   weights = varIdent(form = ~1 | fRegion),
                   correlation=corCAR1(form=~Years),
                   data=PanAfEl16Day, method='ML', control=ctrl)

f_abiotic <- formula(logArea~1 + s.TRMM + s.WATER + s.LST + s.SLOPE)
m_abiotic<-lme(f_abiotic, random=list(fRegion=~1, MovDataID=~1),
               weights = varIdent(form = ~1 | fRegion),
               correlation=corCAR1(form=~Years),
               data=PanAfEl16Day, method='ML', control=ctrl)

f_abiotic_indv <- formula(logArea~1 + Sex + Species + s.TRMM + s.WATER + s.LST + s.SLOPE)
m_abiotic_indv<-lme(f_abiotic_indv, random=list(fRegion=~1, MovDataID=~1),
                    weights = varIdent(form = ~1 | fRegion),
                    correlation=corCAR1(form=~Years),
                    data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic <-formula(logArea~1 + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic<-lme(f_biotic_abiotic, random=list(fRegion=~1, MovDataID=~1),
                      weights = varIdent(form = ~1 | fRegion),
                      correlation=corCAR1(form=~Years),
                      data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic_indv <-formula(logArea~1 + Sex + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic_indv<-lme(f_biotic_abiotic_indv, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic_sex <-formula(logArea~1 + Sex + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic_sex<-lme(f_biotic_abiotic_sex, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic_species <-formula(logArea~1 + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic_species<-lme(f_biotic_abiotic_species, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic_anthro_species <-formula(logArea~1 + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_species<-lme(f_biotic_abiotic_anthro_species, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='ML', control=ctrl)

f_biotic_abiotic_anthro_sex <-formula(logArea~1 + Sex + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_sex<-lme(f_biotic_abiotic_anthro_sex, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='ML', control=ctrl)

f_full <- formula(logArea~1 + Sex + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full<-lme(f_full, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fRegion), 
            correlation=corCAR1(form=~Years), 
            data=PanAfEl16Day, method='ML', control=ctrl)

mod_sel<-model.sel(
    m_sex,
    m_species,
    m_indv,
    m_anthro,
    m_anthro_indv,
    m_biotic,
    m_biotic_indv,
    m_abiotic,
    m_abiotic_indv,
    m_biotic_abiotic,
    m_biotic_abiotic_indv,
    m_biotic_abiotic_sex,
    m_biotic_abiotic_species,
    m_biotic_abiotic_anthro_species,
    m_biotic_abiotic_anthro_sex,
    m_full)

mod_sel # Model m_biotic_abiotic_anthro_species is selected


'''
                                (Int) Sex Spc    s.HFI    s.PAI   s.NDV
m_biotic_abiotic_anthro_species 2.868       + -0.03227 -0.03727 0.03556
m_full                          2.870   +   + -0.03226 -0.03729 0.03556
m_biotic_abiotic_anthro_sex     2.749   +     -0.03184 -0.03781 0.03563
m_biotic_abiotic_species        2.888       +                   0.03461
m_biotic_abiotic                2.773                           0.03468
m_biotic_abiotic_indv           2.890   +   +                   0.03461
m_biotic_abiotic_sex            2.771   +                       0.03468
m_abiotic                       2.776                                  
m_abiotic_indv                  2.889   +   +                          
m_anthro                        2.748         -0.02964 -0.04536        
m_anthro_indv                   2.825   +   + -0.03020 -0.04495        
m_biotic                        2.773                           0.02363
m_species                       2.842       +                          
m_sex                           2.772   +                              
m_biotic_indv                   2.844   +   +                   0.02347
m_indv                          2.844   +   +                          
                                     s.TRE     s.LST    s.SLO    s.TRM  s.WAT
m_biotic_abiotic_anthro_species -0.0011350 0.0212800 -0.01160 0.008969 0.1643
m_full                          -0.0011200 0.0212900 -0.01161 0.008971 0.1644
m_biotic_abiotic_anthro_sex     -0.0044190 0.0210700 -0.01050 0.008825 0.1643
m_biotic_abiotic_species         0.0007031 0.0176500 -0.01621 0.010200 0.1644
m_biotic_abiotic                -0.0026040 0.0174700 -0.01509 0.010050 0.1644
m_biotic_abiotic_indv            0.0007117 0.0176500 -0.01622 0.010210 0.1644
m_biotic_abiotic_sex            -0.0025980 0.0174700 -0.01508 0.010050 0.1644
m_abiotic                                  0.0006545 -0.01411 0.010060 0.1645
m_abiotic_indv                             0.0003190 -0.01445 0.010190 0.1647
m_anthro                                                                     
m_anthro_indv                                                                
m_biotic                        -0.0078180                                   
m_species                                                                    
m_sex                                                                        
m_biotic_indv                   -0.0057050                                   
m_indv                                                                       
                                            family df    logLik    AICc  delta
m_biotic_abiotic_anthro_species gaussian(identity) 32 -9299.146 18662.5   0.00
m_full                          gaussian(identity) 33 -9299.144 18664.5   2.01
m_biotic_abiotic_anthro_sex     gaussian(identity) 32 -9300.393 18665.0   2.49
m_biotic_abiotic_species        gaussian(identity) 30 -9308.374 18676.9  14.43
m_biotic_abiotic                gaussian(identity) 29 -9309.605 18677.4  14.88
m_biotic_abiotic_indv           gaussian(identity) 31 -9308.373 18678.9  16.44
m_biotic_abiotic_sex            gaussian(identity) 30 -9309.603 18679.4  16.89
m_abiotic                       gaussian(identity) 27 -9313.533 18681.2  18.72
m_abiotic_indv                  gaussian(identity) 29 -9312.395 18683.0  20.46
m_anthro                        gaussian(identity) 25 -9492.417 19035.0 372.46
m_anthro_indv                   gaussian(identity) 27 -9491.764 19037.7 375.18
m_biotic                        gaussian(identity) 25 -9500.547 19051.2 388.72
m_species                       gaussian(identity) 24 -9502.226 19052.6 390.07
m_sex                           gaussian(identity) 24 -9502.822 19053.8 391.26
m_biotic_indv                   gaussian(identity) 27 -9499.946 19054.0 391.54
m_indv                          gaussian(identity) 25 -9502.224 19054.6 392.08
                                weight
m_biotic_abiotic_anthro_species  0.604
m_full                           0.221
m_biotic_abiotic_anthro_sex      0.174
m_biotic_abiotic_species         0.000
m_biotic_abiotic                 0.000
m_biotic_abiotic_indv            0.000
m_biotic_abiotic_sex             0.000
m_abiotic                        0.000
m_abiotic_indv                   0.000
m_anthro                         0.000
m_anthro_indv                    0.000
m_biotic                         0.000
m_species                        0.000
m_sex                            0.000
m_biotic_indv                    0.000
m_indv                           0.000
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
'''

In [ ]:
%%R

## Refit with REML
f_biotic_abiotic_anthro_species <-formula(logArea~1 + fSpecies + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_species<-lme(f_biotic_abiotic_anthro_species, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='REML', control=ctrl)

## Create a diagnostic plot of the conditional model
DiagnosticPlot(m_biotic_abiotic_anthro_species, PanAfEl16Day$logArea, PanAfEl16Day$Years,0,
               "../../Figures/16Day_Model_Diagnostics.png")

## Export model summary table
summary(m_biotic_abiotic_anthro_species)
write.csv(summary(m_biotic_abiotic_anthro_species)$tTable, file = "../../Tables/tTable_16Day.csv")

'''
Linear mixed-effects model fit by REML
 Data: PanAfEl16Day 
       AIC      BIC    logLik
  18724.44 18956.15 -9330.221

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:   0.5021581

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.3719902 0.7194702

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
         Phi 
1.713915e-06 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fRegion 
 Parameter estimates:
          APNR    Asante Sana   Chyulu Hills          Coast  Dzanga-Sangha         Gourma         Ivindo         Kruger       Laikipia           Lewa         Loango 
     1.0000000      0.8740580      0.9970340      0.8992496      1.1897613      1.2298731      1.0857930      1.0744045      0.9680075      0.7872732      0.7663703 
          Lope       Marsabit     Masai Mara    Mount Kenya       Northern Nouabale-Ndoki         Odzala        Samburu 
     1.3644978      0.9221654      0.9504148      0.9538801      1.0587643      0.9561824      1.1282823      0.8919332 
Fixed effects: list(f_biotic_abiotic_anthro_species) 
                             Value  Std.Error    DF   t-value p-value
(Intercept)              2.8659540 0.14744735 10082 19.437135  0.0000
fSpeciesForest Elephant -0.4563130 0.29248493    17 -1.560125  0.1372
s.NDVI                   0.0355306 0.01254201 10082  2.832925  0.0046
s.TREE                  -0.0015587 0.01289879 10082 -0.120841  0.9038
s.WATER                  0.1645239 0.00836087 10082 19.677831  0.0000
s.TRMM                   0.0089561 0.00607060 10082  1.475327  0.1402
s.LST                    0.0211690 0.01155403 10082  1.832174  0.0670
s.SLOPE                 -0.0113435 0.01033266 10082 -1.097828  0.2723
s.HFI                   -0.0322079 0.01013008 10082 -3.179438  0.0015
s.PAI                   -0.0373571 0.01250028 10082 -2.988501  0.0028
 Correlation: 
                        (Intr) fSpcFE s.NDVI s.TREE s.WATE s.TRMM s.LST  s.SLOP s.HFI 
fSpeciesForest Elephant -0.502                                                        
s.NDVI                   0.002  0.005                                                 
s.TREE                  -0.011 -0.139 -0.228                                          
s.WATER                  0.015 -0.027 -0.011 -0.001                                   
s.TRMM                   0.002 -0.014 -0.008  0.035 -0.009                            
s.LST                   -0.003 -0.003  0.494  0.081 -0.030  0.135                     
s.SLOPE                 -0.026  0.046  0.004 -0.247 -0.001 -0.025  0.093              
s.HFI                    0.010  0.025 -0.006  0.053 -0.020  0.054 -0.079 -0.128       
s.PAI                    0.038 -0.022 -0.015 -0.010  0.027  0.015 -0.021 -0.017  0.008

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-6.38653643 -0.63041311 -0.01589249  0.61323266  4.13886384 

Number of Observations: 10319
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    19                    229 

'''

## Amount of explained variation (marginal, conditional)
r.squaredGLMM(m_biotic_abiotic_anthro_species)

'''
            R2m       R2c
[1,] 0.03232268 0.4484489
Warning message:
'r.squaredGLMM' now calculates a revised statistic. See the help page. 
'''

myplots <- model_predict(m_biotic_abiotic_anthro_species, dataF=PanAfEl16Day,  yaxis_spacing=5)
png("../../Figures/16Day_Model_Predictions_new2.png", width = 10, height=7, res=300, units='in')
do.call("grid.arrange", c(myplots))
dev.off()

#-------------------------------------------------------------------------------
### Get model summaries for other top models

f_full <- formula(logArea~1 + Sex + Species + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full<-lme(f_full, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fRegion), 
            correlation=corCAR1(form=~Years), 
            data=PanAfEl16Day, method='REML', control=ctrl)
summary(m_full)

'''
Linear mixed-effects model fit by REML
 Data: PanAfEl16Day 
       AIC   BIC    logLik
  18730.05 18969 -9332.027

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:   0.5023377

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.3733526 0.7194919

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
         Phi 
1.715189e-06 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fRegion 
 Parameter estimates:
          APNR    Asante Sana   Chyulu Hills          Coast  Dzanga-Sangha         Gourma         Ivindo         Kruger       Laikipia           Lewa         Loango 
     1.0000000      0.8741717      0.9968651      0.9002842      1.1893316      1.2298694      1.0855842      1.0744214      0.9679591      0.7872526      0.7668488 
          Lope       Marsabit     Masai Mara    Mount Kenya       Northern Nouabale-Ndoki         Odzala        Samburu 
     1.3643014      0.9221668      0.9504260      0.9538899      1.0587472      0.9564952      1.1264494      0.8918706 
Fixed effects: list(f_full) 
                            Value  Std.Error    DF   t-value p-value
(Intercept)             2.8685519 0.15182856 10082 18.893362  0.0000
SexMale                -0.0046579 0.06549945   209 -0.071114  0.9434
SpeciesForest Elephant -0.4581508 0.29358969    17 -1.560514  0.1371
s.NDVI                  0.0355283 0.01254263 10082  2.832605  0.0046
s.TREE                 -0.0015590 0.01290230 10082 -0.120834  0.9038
s.WATER                 0.1645347 0.00836136 10082 19.677989  0.0000
s.TRMM                  0.0089557 0.00607054 10082  1.475271  0.1402
s.LST                   0.0211604 0.01155447 10082  1.831360  0.0671
s.SLOPE                -0.0113359 0.01033635 10082 -1.096707  0.2728
s.HFI                  -0.0321719 0.01013138 10082 -3.175474  0.0015
s.PAI                  -0.0373745 0.01250463 10082 -2.988851  0.0028
 Correlation: 
                       (Intr) SexMal SpcsFE s.NDVI s.TREE s.WATE s.TRMM s.LST  s.SLOP s.HFI 
SexMale                -0.236                                                               
SpeciesForest Elephant -0.505  0.079                                                        
s.NDVI                  0.003 -0.004  0.005                                                 
s.TREE                 -0.007 -0.016 -0.140 -0.228                                          
s.WATER                 0.013  0.005 -0.026 -0.011 -0.001                                   
s.TRMM                  0.002  0.000 -0.014 -0.008  0.035 -0.009                            
s.LST                  -0.004  0.001 -0.003  0.494  0.081 -0.030  0.135                     
s.SLOPE                -0.031  0.024  0.047  0.004 -0.247 -0.001 -0.025  0.093              
s.HFI                   0.013 -0.011  0.024 -0.006  0.053 -0.020  0.054 -0.079 -0.129       
s.PAI                   0.032  0.021 -0.020 -0.015 -0.010  0.027  0.015 -0.021 -0.017  0.008

Standardized Within-Group Residuals:
       Min         Q1        Med         Q3        Max 
-6.3865848 -0.6303741 -0.0160936  0.6127268  4.1392038 

Number of Observations: 10319
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    19                    229 
'''

f_biotic_abiotic_anthro_sex <-formula(logArea~1 + Sex + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_sex<-lme(f_biotic_abiotic_anthro_sex, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fRegion),
                           correlation=corCAR1(form=~Years),
                           data=PanAfEl16Day, method='REML', control=ctrl)
summary(m_biotic_abiotic_anthro_sex)

'''
Linear mixed-effects model fit by REML
 Data: PanAfEl16Day 
       AIC      BIC   logLik
  18729.72 18961.43 -9332.86

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:   0.5405705

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.3727141 0.7194288

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
         Phi 
1.702386e-06 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fRegion 
 Parameter estimates:
          APNR    Asante Sana   Chyulu Hills          Coast  Dzanga-Sangha         Gourma         Ivindo         Kruger       Laikipia           Lewa         Loango 
     1.0000000      0.8730920      0.9968061      0.9026468      1.1804900      1.2296425      1.0859116      1.0743150      0.9678144      0.7874126      0.7675898 
          Lope       Marsabit     Masai Mara    Mount Kenya       Northern Nouabale-Ndoki         Odzala        Samburu 
     1.3650608      0.9221470      0.9504166      0.9536880      1.0588261      0.9363118      1.1372466      0.8917197 
Fixed effects: list(f_biotic_abiotic_anthro_sex) 
                 Value  Std.Error    DF   t-value p-value
(Intercept)  2.7487732 0.13899283 10082 19.776368  0.0000
SexMale      0.0022779 0.06528185   209  0.034893  0.9722
s.NDVI       0.0355984 0.01254264 10082  2.838190  0.0045
s.TREE      -0.0044692 0.01279344 10082 -0.349337  0.7268
s.WATER      0.1644114 0.00836273 10082 19.660010  0.0000
s.TRMM       0.0088256 0.00606999 10082  1.453974  0.1460
s.LST        0.0210052 0.01155444 10082  1.817936  0.0691
s.SLOPE     -0.0104193 0.01032795 10082 -1.008848  0.3131
s.HFI       -0.0318067 0.01012892 10082 -3.140190  0.0017
s.PAI       -0.0378116 0.01250631 10082 -3.023399  0.0025
 Correlation: 
        (Intr) SexMal s.NDVI s.TREE s.WATE s.TRMM s.LST  s.SLOP s.HFI 
SexMale -0.214                                                        
s.NDVI   0.005 -0.004                                                 
s.TREE  -0.088 -0.006 -0.229                                          
s.WATER  0.000  0.007 -0.011 -0.005                                   
s.TRMM  -0.006  0.001 -0.008  0.033 -0.009                            
s.LST   -0.006  0.001  0.494  0.082 -0.030  0.135                     
s.SLOPE -0.007  0.020  0.004 -0.243  0.001 -0.024  0.093              
s.HFI    0.028 -0.013 -0.006  0.057 -0.020  0.055 -0.079 -0.130       
s.PAI    0.024  0.022 -0.015 -0.013  0.026  0.015 -0.021 -0.016  0.008

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-6.38940786 -0.62963180 -0.01551716  0.61270068  4.13889000 

Number of Observations: 10319
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    19                    229 
'''

## Annual Range Model

In [ ]:
%%R

## Modeling procedure based on 'Top-Down' described by West et al 2014 pg. 39.

## STEP 1: Start with a well-specified mean structure

# Colinearity between s.NDVI, s.EVI, s.LST, s.TREE  so need to choose best full model

# s.NDVI model
f_full_s.NDVI <- formula(logArea~1 + fSex + fSpecies + s.NDVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE + s.HFI + s.PAI)
m_full_s.NDVI<-lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                   data=PanAfElAnnual, method='REML', control=ctrl)

# s.EVI model
f_full_s.EVI <- formula(logArea~1 + fSex + fSpecies + s.EVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE + s.HFI + s.PAI)
m_full_s.EVI<-lme(f_full_s.EVI, random=list(fRegion=~1, MovDataID=~1),
                  data=PanAfElAnnual, method='REML', control=ctrl)

# s.TEMP model
f_full_s.LST <- formula(logArea~1 + fSex + fSpecies + s.NDWI + s.WATER + s.TRMM + s.LST + s.SLOPE + s.HFI + s.PAI)
m_full_s.LST<-lme(f_full_s.LST, random=list(fRegion=~1, MovDataID=~1),
                   data=PanAfElAnnual, method='REML', control=ctrl)

# s.TREE model
f_full_s.TREE <- formula(logArea~1 + fSex + fSpecies + s.NDWI + s.WATER + s.TRMM + s.TREE + s.SLOPE + s.HFI + s.PAI)
m_full_s.TREE<-lme(f_full_s.TREE, random=list(fRegion=~1, MovDataID=~1),
                   data=PanAfElAnnual, method='REML', control=ctrl)

results<-model.sel(m_full_s.NDVI, m_full_s.EVI, m_full_s.LST, m_full_s.TREE)
results # Model selection favours s.NDVI. Adopt s.NDVI full model.

'''
Model selection table 
              (Int) fSx fSp   s.HFI  s.NDV    s.NDW   s.PAI     s.SLO   s.TRM     s.WAT   s.EVI    s.LST    s.TRE             family df   logLik  AICc delta weight
m_full_s.NDVI 5.086   +   + -0.1135 0.0744 -0.09653 -0.1186 -0.024610 0.06013 -0.000722                           gaussian(identity) 13 -200.600 428.5  0.00  0.309
m_full_s.EVI  5.084   +   + -0.1167        -0.09693 -0.1201 -0.020130 0.06096  0.002744 0.06571                   gaussian(identity) 13 -200.683 428.6  0.17  0.285
m_full_s.TREE 5.053   +   + -0.1216        -0.03498 -0.1140 -0.002573 0.07316  0.003065                  -0.04456 gaussian(identity) 13 -200.970 429.2  0.74  0.214
m_full_s.LST  5.072   +   + -0.1134        -0.06636 -0.1163 -0.025670 0.06578 -0.003081         -0.03686          gaussian(identity) 13 -201.075 429.4  0.95  0.192
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
'''

## Select a structure for the random effects
m_gls_0 <- gls(f_full_s.NDVI, data=PanAfElAnnual, method='REML') # See pg. 93 of West et al. 
m_rand_1 <- lme(f_full_s.NDVI, random=~1|fRegion, data=PanAfElAnnual, method='REML')
m_rand_2 <- lme(f_full_s.NDVI, random=~1|MovDataID, data=PanAfElAnnual, method='REML')
m_rand_3 <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1), data=PanAfElAnnual, method='REML')
results <-model.sel(m_gls_0, m_rand_1, m_rand_2, m_rand_3)
results # Model selection favours m_rand_3
'''
Model selection table 
         (Int) fSx fSp   s.HFI    s.NDV    s.NDW      s.PAI     s.SLO    s.TRM      s.WAT             family class  random df   logLik  AICc  delta weight
m_rand_3 5.086   +   + -0.1135  0.07440 -0.09653 -0.1186000 -0.024610  0.06013 -0.0007221 gaussian(identity)   lme f+M%i%f 13 -200.600 428.5   0.00      1
m_rand_2 5.180   +   + -0.1232 -0.01306  0.06472  0.0008957 -0.174100 -0.05536 -0.0044040 gaussian(identity)   lme       M 12 -221.056 467.2  38.73      0
m_rand_1 5.004   +   + -0.2255  0.02670 -0.10570 -0.1104000  0.006952  0.10790 -0.0576000 gaussian(identity)   lme       f 12 -251.398 527.9  99.41      0
m_gls_0  5.131   +   + -0.2167 -0.01360  0.05267 -0.0252100 -0.209600 -0.12630 -0.0252100 gaussian(identity)   gls         11 -280.939 584.8 156.32      0
Models ranked by AICc(x) 
Random terms: 
f = ‘1 | fRegion’
M%i%f = ‘1 | MovDataID %in% fRegion’
M = ‘1 | MovDataID’
'''

## Select a correlation structure for the residuals
m_rand_3_cor <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                    correlation=corCAR1(form=~Years),
                    data=PanAfElAnnual, method='REML', control=ctrl)
results <-model.sel(m_rand_3, m_rand_3_cor)
results #  m_rand_3_cor is selected

"""
Model selection table 
             (Int) fSx fSp   s.HFI   s.NDV    s.NDW   s.PAI    s.SLO   s.TRM      s.WAT             family     correlation control df   logLik  AICc delta weight
m_rand_3_cor 5.080   +   + -0.1044 0.05507 -0.11060 -0.1200 -0.01411 0.07673 -0.0196800 gaussian(identity) corCAR1(~Years)    ctrl 14 -196.781 423.0  0.00  0.938
m_rand_3     5.086   +   + -0.1135 0.07440 -0.09653 -0.1186 -0.02461 0.06013 -0.0007221 gaussian(identity)                         13 -200.600 428.5  5.44  0.062
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
"""

## select a covariance structure for the residuals
m_rand_3_cor_weights_none <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                 correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
m_rand_3_cor_weights_region <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                   weights = varIdent(form = ~1 | fRegion),
                                   correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
m_rand_3_cor_weights_sex <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                weights = varIdent(form = ~1 | fSex),
                                correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
m_rand_3_cor_weights_species <- lme(f_full_s.NDVI, random=list(fRegion=~1, MovDataID=~1),
                                    weights = varIdent(form = ~1 | fSpecies),
                                    correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
results <-model.sel(m_rand_3_cor_weights_none, m_rand_3_cor_weights_region,
                    m_rand_3_cor_weights_sex, m_rand_3_cor_weights_species)

results # Model m_rand_3_cor_weights_sex is selected
"""
Model selection table 
                             (Int) fSx fSp    s.HFI   s.NDV    s.NDW   s.PAI    s.SLO   s.TRM      s.WAT             family               weights df   logLik  AICc
m_rand_3_cor_weights_sex     5.084   +   + -0.11200 0.06426 -0.10390 -0.1496 -0.01049 0.06562  8.144e-05 gaussian(identity)     varIdent(~1|fSex) 15 -191.040 413.8
m_rand_3_cor_weights_region  5.064   +   + -0.02701 0.12690 -0.10700 -0.2191 -0.02329 0.05889  9.337e-03 gaussian(identity)  varIdent(~1|fRegion) 27 -178.647 416.8
m_rand_3_cor_weights_species 5.063   +   + -0.10740 0.03474 -0.09841 -0.1367 -0.01214 0.07841 -1.567e-02 gaussian(identity) varIdent(~1|fSpecies) 15 -194.229 420.1
m_rand_3_cor_weights_none    5.080   +   + -0.10440 0.05507 -0.11060 -0.1200 -0.01411 0.07673 -1.968e-02 gaussian(identity)                       14 -196.781 423.0
                             delta weight
m_rand_3_cor_weights_sex      0.00  0.789
m_rand_3_cor_weights_region   3.05  0.171
m_rand_3_cor_weights_species  6.38  0.033
m_rand_3_cor_weights_none     9.27  0.008
Models ranked by AICc(x) 
Random terms (all models): 
‘1 | fRegion’, ‘1 | MovDataID %in% fRegion’
"""

In [ ]:
%%R

## Choose fixed effects with and without sex species interactions
f_indv <- formula(logArea~1 + fSex + fSpecies)
m_indv<-lme(f_indv, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_anthro <- formula(logArea~1 + s.HFI + s.PAI)
m_anthro <-lme(f_anthro, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_anthro_indv <-formula(logArea~1 + fSex + fSpecies + s.HFI + s.PAI)
m_anthro_indv<-lme(f_anthro_indv, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_biotic <- formula(logArea~1 + s.NDVI + s.NDWI)
m_biotic<-lme(f_biotic, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_biotic_indv <- formula(logArea~1 + fSex + fSpecies + s.NDVI + s.NDWI)
m_biotic_indv<-lme(f_biotic_indv, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_abiotic <- formula(logArea~1 + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_abiotic<-lme(f_abiotic, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_abiotic_indv <- formula(logArea~1 + fSex + fSpecies + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_abiotic_indv<-lme(f_abiotic_indv, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_biotic_abiotic <-formula(logArea~1 + s.NDVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE)
m_biotic_abiotic<-lme(f_biotic_abiotic, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_biotic_abiotic_indv <-formula(logArea~1 + fSex + fSpecies + s.NDVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE)
m_biotic_abiotic_indv<-lme(f_biotic_abiotic_indv, random=list(fRegion=~1, MovDataID=~1), 
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

f_biotic_abiotic_sex <-formula(logArea~1 + fSex + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic_sex<-lme(f_biotic_abiotic_sex, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fSex),
                           correlation=corCAR1(form=~Years),
                           data=PanAfElAnnual, method='ML')

f_biotic_abiotic_species <-formula(logArea~1 + fSpecies + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE)
m_biotic_abiotic_species<-lme(f_biotic_abiotic_species, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fSex),
                           correlation=corCAR1(form=~Years),
                           data=PanAfElAnnual, method='ML')

f_biotic_abiotic_anthro_species <-formula(logArea~1 + fSpecies + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_species<-lme(f_biotic_abiotic_anthro_species, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fSex),
                           correlation=corCAR1(form=~Years),
                           data=PanAfElAnnual, method='ML')

f_biotic_abiotic_anthro_sex <-formula(logArea~1 + fSex + s.NDVI + s.TREE + s.WATER + s.TRMM + s.LST + s.SLOPE +
                                          s.HFI + s.PAI)
m_biotic_abiotic_anthro_sex<-lme(f_biotic_abiotic_anthro_sex, random=list(fRegion=~1, MovDataID=~1),
                           weights = varIdent(form = ~1 | fSex),
                           correlation=corCAR1(form=~Years),
                           data=PanAfElAnnual, method='ML')

f_full <- formula(logArea~1 + fSex + fSpecies + s.NDVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE + s.HFI + s.PAI)
m_full<-lme(f_full, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='ML')

mod_sel<-model.sel(m_indv, m_anthro, m_anthro_indv, m_biotic, m_biotic_indv, m_abiotic,
                   m_abiotic_indv, m_biotic_abiotic, m_biotic_abiotic_indv,
                   m_biotic_abiotic_sex,
                   m_biotic_abiotic_species,
                   m_biotic_abiotic_anthro_species,
                   m_biotic_abiotic_anthro_sex,
                   m_full)

mod_sel # m_anthro_indv is selected
'''
Model selection table 
                                (Int) fSx fSp    s.HFI   s.PAI    s.NDV    s.NDW     s.LST    s.SLO    s.TRM      s.WAT    s.TRE             family df   logLik
m_anthro_indv                   5.036   +   + -0.11960 -0.1362                                                                   gaussian(identity) 10 -176.532
m_anthro                        4.841         -0.09608 -0.1479                                                                   gaussian(identity)  8 -180.566
m_full                          5.087   +   + -0.11360 -0.1477  0.06165 -0.09935           -0.01324 0.063380  0.0008561          gaussian(identity) 15 -174.390
m_biotic_abiotic_anthro_species 5.106       + -0.12480 -0.1368  0.04569           0.004346 -0.02013 0.037720  0.0092450 -0.05528 gaussian(identity) 15 -175.804
m_indv                          5.075   +   +                                                                                    gaussian(identity)  8 -183.733
m_biotic_indv                   5.124   +   +                   0.07672 -0.10140                                                 gaussian(identity) 10 -182.646
m_biotic_abiotic_anthro_sex     4.840   +     -0.10830 -0.1452  0.03994          -0.005716  0.00565 0.028080 -0.0183100 -0.10420 gaussian(identity) 15 -177.787
m_biotic                        4.971                           0.07156 -0.11720                                                 gaussian(identity)  8 -185.397
m_biotic_abiotic_indv           5.150   +   +                   0.06682 -0.12050           -0.04325 0.048270 -0.0086260          gaussian(identity) 13 -181.388
m_abiotic_indv                  5.107   +   +                                    -0.023830 -0.06399 0.019800 -0.0011180          gaussian(identity) 12 -182.719
m_biotic_abiotic                4.989                           0.06093 -0.14070           -0.03440 0.043360 -0.0373900          gaussian(identity) 11 -184.080
m_biotic_abiotic_species        5.150       +                  -0.01963          -0.061050 -0.05984 0.018840 -0.0071780 -0.05438 gaussian(identity) 13 -182.640
m_abiotic                       4.924                                            -0.015050 -0.05670 0.006071 -0.0304700          gaussian(identity) 10 -186.067
m_biotic_abiotic_sex            4.949   +                      -0.01681          -0.063350 -0.03462 0.009664 -0.0288700 -0.10330 gaussian(identity) 13 -184.055
                                 AICc delta weight
m_anthro_indv                   373.8  0.00  0.829
m_anthro                        377.6  3.80  0.124
m_full                          380.5  6.64  0.030
m_biotic_abiotic_anthro_species 383.3  9.47  0.007
m_indv                          384.0 10.14  0.005
m_biotic_indv                   386.0 12.23  0.002
m_biotic_abiotic_anthro_sex     387.3 13.43  0.001
m_biotic                        387.3 13.47  0.001
m_biotic_abiotic_indv           390.0 16.22  0.000
m_abiotic_indv                  390.5 16.70  0.000
m_biotic_abiotic                391.1 17.25  0.000
m_biotic_abiotic_species        392.5 18.73  0.000
m_abiotic                       392.9 19.07  0.000
m_biotic_abiotic_sex            395.4 21.55  0.000
'''

In [ ]:
%%R

## Refit with REML
f_anthro_indv <-formula(logArea~1 + fSex + fSpecies + s.HFI + s.PAI)
m_anthro_indv <-lme(f_anthro_indv, random=list(fRegion=~1, MovDataID=~1),
                         weights = varIdent(form = ~1 | fSex),
                         correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')

## Create a diagnostic plot of the population model
DiagnosticPlot(m_anthro_indv, PanAfElAnnual$logArea,PanAfElAnnual$Years,0, "../../Figures/Annual_Model_Diagnostics.png")

## Export model summary table
summary(m_anthro_indv)
write.csv(summary(m_anthro_indv)$tTable, file = "../../Tables/tTable_Annual.csv")

'''
Linear mixed-effects model fit by REML
 Data: PanAfElAnnual 
       AIC      BIC    logLik
  385.7162 422.6535 -182.8581

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:    0.653593

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.4121123 0.3866323

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
      Phi 
0.3504766 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fSex 
 Parameter estimates:
     Male    Female 
1.0000000 0.6727519 
Fixed effects: list(f_anthro_indv) 
                            Value Std.Error  DF   t-value p-value
(Intercept)              5.032861 0.2103820 171 23.922487  0.0000
fSexMale                 0.115915 0.0960162 114  1.207241  0.2298
fSpeciesForest Elephant -1.299569 0.4914727  12 -2.644235  0.0214
s.HFI                   -0.117579 0.0440601 171 -2.668598  0.0084
s.PAI                   -0.138168 0.0527692 171 -2.618353  0.0096
 Correlation: 
                        (Intr) fSexMl fSpcFE s.HFI 
fSexMale                -0.211                     
fSpeciesForest Elephant -0.418  0.040              
s.HFI                    0.011 -0.041  0.182       
s.PAI                    0.053 -0.008 -0.056 -0.104

Standardized Within-Group Residuals:
         Min           Q1          Med           Q3          Max 
-3.324456320 -0.553053626 -0.005699585  0.553857061  2.542789138 

Number of Observations: 302
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    14                    129 

'''

## Amount of explained variation (marginal, conditional)
r.squaredGLMM(m_anthro_indv)

'''
            R2m       R2c
[1,] 0.06799563 0.8133699
Warning message:
'r.squaredGLMM' now calculates a revised statistic. See the help page. 

'''

## Generate prediction plots
myplots <- model_predict(m_anthro_indv, dataF=PanAfElAnnual, yaxis_spacing=50)

png("../../Figures/Annual_Model_Predictions_new2.png", width = 7, height = 10, res=300, units='in')
do.call("grid.arrange", c(myplots))
dev.off()

# Get parameter values of other top two models------------------------------

f_anthro <- formula(logArea~1 + s.HFI + s.PAI)
m_anthro <-lme(f_anthro, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
summary(m_anthro)

'''
Linear mixed-effects model fit by REML
 Data: PanAfElAnnual 
       AIC      BIC    logLik
  386.7548 416.3584 -185.3774

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:   0.8227071

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.4149744 0.3887844

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
      Phi 
0.3613904 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fSex 
 Parameter estimates:
     Male    Female 
1.0000000 0.6676575 
Fixed effects: list(f_anthro) 
                Value  Std.Error  DF   t-value p-value
(Intercept)  4.839022 0.23004432 171 21.035172  0.0000
s.HFI       -0.096757 0.04364868 171 -2.216725  0.0280
s.PAI       -0.147901 0.05324849 171 -2.777570  0.0061
 Correlation: 
      (Intr) s.HFI 
s.HFI  0.077       
s.PAI  0.026 -0.098

Standardized Within-Group Residuals:
         Min           Q1          Med           Q3          Max 
-3.301044073 -0.526112712  0.003942935  0.545517391  2.586206558 

Number of Observations: 302
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    14                    129 
'''

f_full <- formula(logArea~1 + fSex + fSpecies + s.NDVI + s.NDWI + s.WATER + s.TRMM + s.SLOPE + s.HFI + s.PAI)
m_full<-lme(f_full, random=list(fRegion=~1, MovDataID=~1),
            weights = varIdent(form = ~1 | fSex), correlation=corCAR1(form=~Years), data=PanAfElAnnual, method='REML')
summary(m_full)

'''
Linear mixed-effects model fit by REML
 Data: PanAfElAnnual 
       AIC      BIC    logLik
  412.0806 467.2319 -191.0403

Random effects:
 Formula: ~1 | fRegion
        (Intercept)
StdDev:   0.7134374

 Formula: ~1 | MovDataID %in% fRegion
        (Intercept)  Residual
StdDev:   0.4041972 0.3904173

Correlation Structure: Continuous AR(1)
 Formula: ~Years | fRegion/MovDataID 
 Parameter estimate(s):
      Phi 
0.3754444 
Variance function:
 Structure: Different standard deviations per stratum
 Formula: ~1 | fSex 
 Parameter estimates:
     Male    Female 
1.0000000 0.6816292 
Fixed effects: list(f_full) 
                              Value Std.Error  DF   t-value p-value
(Intercept)                3.663349 0.5299611 166  6.912486  0.0000
fSexMale                   0.104592 0.0972465 114  1.075532  0.2844
fSpeciesSavannah Elephant  1.420974 0.5772171  12  2.461768  0.0299
s.NDVI                     0.064265 0.0743832 166  0.863970  0.3889
s.NDWI                    -0.103932 0.0708637 166 -1.466646  0.1444
s.WATER                    0.000081 0.0452679 166  0.001799  0.9986
s.TRMM                     0.065623 0.0432805 166  1.516234  0.1314
s.SLOPE                   -0.010493 0.0515302 166 -0.203623  0.8389
s.HFI                     -0.111966 0.0462861 166 -2.418993  0.0166
s.PAI                     -0.149623 0.0536288 166 -2.789970  0.0059
 Correlation: 
                          (Intr) fSexMl fSpcSE s.NDVI s.NDWI s.WATE s.TRMM s.SLOP s.HFI 
fSexMale                  -0.073                                                        
fSpeciesSavannah Elephant -0.916 -0.019                                                 
s.NDVI                    -0.031 -0.109  0.076                                          
s.NDWI                    -0.166  0.095  0.099 -0.577                                   
s.WATER                   -0.326  0.042  0.322 -0.068  0.142                            
s.TRMM                    -0.110  0.055  0.102 -0.340 -0.196  0.135                     
s.SLOPE                    0.028  0.157 -0.070 -0.303 -0.050  0.013  0.251              
s.HFI                      0.268 -0.084 -0.228  0.091 -0.204 -0.115 -0.072 -0.103       
s.PAI                     -0.041 -0.016  0.059 -0.051  0.081  0.075 -0.069 -0.095 -0.096

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-3.25677074 -0.52655069 -0.02966863  0.52370330  2.53946504 

Number of Observations: 302
Number of Groups: 
               fRegion MovDataID %in% fRegion 
                    14                    129 
'''

# Habitat Suitability Model

### Convert GCS to Albers to get aereal values

In [ ]:
arcpy.CreateFileGDB_management ('Y:/Research_PanAfEl/Analyses/HSM', 'HSM')
arcpy.env.workspace ='C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM' + '/HSM.gdb'

In [ ]:
#Set arcpy to overwrite output
arcpy.env.overwriteOutput = True

if arcpy.CheckExtension("Spatial") == "Available":
        arcpy.CheckOutExtension("Spatial")

In [ ]:
## Continental Africa (without Madagascar) - Albers Equal Area Conic
arcpy.Project_management(in_dataset="C:/Users/jake_/Dropbox/Research_PanAfEl/Spatial Data/SpatialData.gdb/ContinentalAfrica",
                         out_dataset="ContinentalAfrica_albers",
                         out_coor_system="PROJCS['Africa_Albers_Equal_Area_Conic',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',25.0],PARAMETER['Standard_Parallel_1',20.0],PARAMETER['Standard_Parallel_2',-23.0],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]",
                         transform_method="",
                         in_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
                         preserve_shape="NO_PRESERVE_SHAPE",
                         max_deviation="",
                         vertical="NO_VERTICAL")

## AfElSG Known + Possible Range
arcpy.Project_management(in_dataset="C:/Users/jake_/Dropbox/Research_PanAfEl/Spatial Data/SpatialData.gdb/AfElSg_Range_Dissolv",
                         out_dataset="afelsg_albers",
                         out_coor_system="PROJCS['Africa_Albers_Equal_Area_Conic',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',25.0],PARAMETER['Standard_Parallel_1',20.0],PARAMETER['Standard_Parallel_2',-23.0],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]",
                         transform_method="",
                         in_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
                         preserve_shape="NO_PRESERVE_SHAPE",
                         max_deviation="",
                         vertical="NO_VERTICAL")

## WDPA
arcpy.Clip_analysis(in_features="C:/Users/jake_/Dropbox/Research_PanAfEl/Spatial Data/WDPA_August2013.gdb/WDPApoly_August2013",
                    clip_features="Continental Africa",
                    out_feature_class="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/WDPA",
                    cluster_tolerance="")

arcpy.Project_management(in_dataset="WDPA",
                         out_dataset="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/WDPA_Albers",
                         out_coor_system="PROJCS['Africa_Albers_Equal_Area_Conic',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',25.0],PARAMETER['Standard_Parallel_1',20.0],PARAMETER['Standard_Parallel_2',-23.0],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]",
                         transform_method="",
                         in_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
                         preserve_shape="NO_PRESERVE_SHAPE",
                         max_deviation="",
                         vertical="NO_VERTICAL")

arcpy.Dissolve_management(in_features="WDPA_Albers",
                          out_feature_class="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/WDPA_Albers_dissolv",
                          dissolve_field="", statistics_fields="", multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

In [ ]:
# Calculate AFELSG Area outside WDPA
arcpy.Erase_analysis(in_features="afelsg_albers",
                     erase_features="WDPA_Albers_dissolv",
                     out_feature_class="afelsg_albers_outside_wdpa",
                     cluster_tolerance="")

In [ ]:
# HSM_Dispersal: - Set Zeroes to Null
arcpy.gp.SetNull_sa("C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/PanAfEl_HSM_Dispersal.tif",
                    "C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/PanAfEl_HSM_Dispersal.tif",
                    "C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM_Dispersal_SetNull.tif",
                    '"Value" = 0')

# Convert HSM_Dispersal Raster to Polygon
arcpy.RasterToPolygon_conversion(in_raster="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM_Dispersal_SetNull.tif",
                                 out_polygon_features="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/HSM_Dispersal_Poly",
                                 simplify="SIMPLIFY",
                                 raster_field="Value",
                                 create_multipart_features="MULTIPLE_OUTER_PART",
                                 max_vertices_per_feature="")

# Project the Dispersal Range to Albers
arcpy.Project_management(in_dataset="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/HSM_Dispersal_Poly", 
                         out_dataset="C:/Users/jake_/Dropbox/Research_PanAfEl/Analyses/HSM/HSM.gdb/HSM_Dispersal_Poly_Albers", 
                         out_coor_system="PROJCS['Africa_Albers_Equal_Area_Conic',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',25.0],PARAMETER['Standard_Parallel_1',20.0],PARAMETER['Standard_Parallel_2',-23.0],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]",
                         transform_method="", 
                         in_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
                         preserve_shape="NO_PRESERVE_SHAPE",
                         max_deviation="",
                         vertical="NO_VERTICAL")

In [ ]:
# Calculate HSM outside of WDPA
arcpy.Erase_analysis(in_features="HSM_Dispersal_Poly_Albers",
                     erase_features="WDPA_Albers_dissolv",
                     out_feature_class="hsm_dispersal_albers_outside_wdpa",
                     cluster_tolerance="")

#### Read dispersal, core and AfElSG range areal values

In [ ]:
# Read these values in from ArcMAP
hsm_dispersal_area = 18169219021690.476563  / 1000000
afelsg_K_P_area = 3129540087056.136719  / 1000000
africa_area = 29282962929907.191406  / 1000000
afelsg_K_P_area_outside_wdpa = 1795945605785.407715  / 1000000
hsm_dispersal_area_outside_wdpa = 15458775026175.25  / 1000000

print('HSM Dispersal Area km2: {0}'.format(hsm_dispersal_area))
print('AfElSG Area km2: {0}'.format(afelsg_K_P_area))
print('Continental Africa Area km2: {0}'.format(africa_area))
print('AfElSG Area Outside WDPA km2: {0}'.format(hsm_dispersal_area_outside_wdpa))
print('HSM Dispersal Area Outside WDPA km2: {0}'.format(hsm_dispersal_area_outside_wdpa))
print('AfElSG area fraction of HSM: {0}%'.format((afelsg_K_P_area / hsm_dispersal_area)*100))
print('Fraction of HSM to African Continent Area: {0}%'.format((hsm_dispersal_area /africa_area)*100))
print('Fraction of AfElSG to African Continent Area: {0}%'.format((afelsg_K_P_area / africa_area)*100))
print('Fraction of HSM Outside of WDPA to overall HSM Area: {0}%'.format((hsm_dispersal_area_outside_wdpa / hsm_dispersal_area)*100))
print('Fraction of AfElSG Outside of WDPA to overall AfElSG Area: {0}%'.format((afelsg_K_P_area_outside_wdpa / afelsg_K_P_area)*100))